# FLIMLib Ops

## Dependencies

The FLIMJ ops live in `flimlib:flimj-ops`. This dependency has to be present in order to use the ops. You can either import the package from your maven local or the SciJava public repository.

In [1]:
// uncomment to import from local repo
%classpath config resolver mvnLocal
// import from SciJava public repo
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
// uncomment to import from SciJava public repo
// %classpath add mvn flimlib flimj-ops 2.1.2
%classpath add mvn org.scijava scijava-table 0.4.0
%classpath add mvn net.imglib2 imglib2 6.2.0
%classpath add mvn io.scif scifio-lifesci 0.9.0
%classpath add mvn org.scijava scijava-ops-flim 0-SNAPSHOT
%classpath add mvn net.imagej imagej 2.0.0-rc-72

import net.imagej.ImageJ
import org.scijava.ops.api.OpEnvironment

ij = new ImageJ()
op = OpEnvironment.build()
nb = ij.notebook()

Added new repo: mvnLocal
Added new repo: scijava.public


net.imagej.notebook.DefaultNotebookService [priority = 0.0]

In [2]:
// run this if dependency messed up
// %classpath reset

null

## Utility Code

Here is some utility code that helps display the multi-layer fitted images, no attention needed.

In [3]:
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.real.FloatType
import net.imagej.display.ColorTables
import net.imglib2.converter.Converters
import net.imglib2.converter.RealLUTConverter

class FancyDisplay {
    
    public channelAxis, op, nb
    
    public FancyDisplay(ij, op, channelAxis=3) {
        this.channelAxis = channelAxis
        this.op = op
        this.nb = ij.notebook()
    }
    
    public tableDisp(fitted, tMin=null, tMax=null, aMax=null, zMax=null) {
        def lifetimeAxis = fitted.ltAxis
        def fittedImg = fitted.paramMap
        def sampleZ = op.ternary("transform.hyperSliceView").input(fittedImg, lifetimeAxis, (long) 0).apply()
        println(sampleZ[0])
        def sampleA = []
        def sampleT = []
        for (int comp in 0..((fittedImg.dimension(lifetimeAxis) - 1) / 2 - 1)) {
            sampleA.push(op.ternary("transform.hyperSliceView").input(fittedImg, lifetimeAxis, (long) (comp * 2 + 1)).apply())
            sampleT.push(op.ternary("transform.hyperSliceView").input(fittedImg, lifetimeAxis, (long) (comp * 2 + 2)).apply())
        }

        def out = new FloatType()
        op.unary("stats.min").input(sampleZ).output(out).compute()
        println("Z min = " + out)
        op.unary("stats.max").input(sampleZ).output(out).compute()
        println("Z max = " + out)
        for (int i in 0..sampleA.size() - 1) {
            op.unary("stats.min").input(sampleA[i]).output(out).compute()
            println("A" + (i + 1) + " min = " + out)
            op.unary("stats.max").input(sampleA[i]).output(out).compute()
            println("A" + (i + 1) + " max = " + out)
            op.unary("stats.min").input(sampleT[i]).output(out).compute()
            println("Tau" + (i + 1) + " min = " + out)
            op.unary("stats.max").input(sampleT[i]).output(out).compute()
            println("Tau" + (i + 1) + " max = " + out)
        }
        
        def pseudocolor = op.op("flim.pseudocolor").input(fitted, (Float) tMin, (Float) tMax, (Float) 0.0, (Float) aMax).apply();
        
        // default values from img
        zMax = zMax == null ? op.unary("stats.max").input(sampleZ).outType(FloatType.class).apply() : new FloatType(zMax)
        aMax = aMax == null ? op.unary("stats.max").input(sampleA[0]).outType(FloatType.class).apply() : new FloatType(aMax)
        tMin = tMin == null ? op.unary("stats.min").input(sampleT[0]).outType(FloatType.class).apply() : new FloatType(tMin)
        tMax = tMax == null ? op.unary("stats.max").input(sampleT[0]).outType(FloatType.class).apply() : new FloatType(tMax)
        
        def labeled = [:]
        labeled["Z"] = nb.display(sampleZ, 0, zMax.getRealFloat())
        
        for (int i in 0..sampleA.size() - 1) {
            labeled["A"   + (i + 1)] = nb.display(sampleA[i], 0, aMax.getRealFloat())
            labeled["Tau" + (i + 1)] = nb.display(sampleT[i], tMin.getRealFloat(), tMax.getRealFloat())
            labeled["Pseudocolor" + (i + 1)] = op.ternary("transform.hyperSliceView").input(pseudocolor, lifetimeAxis, (long) i).apply()
        }
        return [labeled]
    }
}
import org.scijava.ops.api.OpEnvironment
op = OpEnvironment.build()
fcd = new FancyDisplay(ij, op)

FancyDisplay@1ac6e444

## Loading Dataset

Here we use the [scifio](https://imagej.net/SCIFIO) [bio-formats](https://imagej.net/Bio-Formats) plugin to load time-resolved transient data from `input.sdt`.

In [4]:
import org.scijava.io.location.FileLocation
sdtPath = new FileLocation("../test_files/input.sdt")

sdt = ij.scifio().datasetIO().open(sdtPath)

[INFO] Reading SDT header


<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAABOklEQVR4Xu3IMQEAAAyDsPo3vQlAAhx5sm0XNURcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBxAURF0RcEHFBROQBhhDDspG38D4AAAAASUVORK5CYII=" />

The acquired dataset is actually a 4-dimensional image as we will be shown bellow. It appears purely dark because the notebook by default displays the first layer it sees.<br>
We now use the following snippet to "chop up" the dataset for demonstration. We also display the metadata for reference.

In [5]:
import io.scif.lifesci.SDTFormat


sdtReader = new SDTFormat.Reader()
sdtReader.setContext(ij.getContext())
sdtReader.setSource(sdtPath)
sdtMetadata = sdtReader.getMetadata()

// display the axis type of each dimension
for (d = 0; d < sdt.numDimensions(); d++) {
    printf("Dim #%d: size: %3d, type: %s\n", d, sdt.dimension(d), sdt.axis(d).type())
}

timeBase = sdtMetadata.getTimeBase()
timeBins = sdtMetadata.getTimeBins()

printf("Time base: %6f, number of bins: %d\n", timeBase, timeBins)

cStart = 6
cEnd = 15
tStart = 5
tEnd = 16

table = []
for (c in (cStart..cEnd)) {
    row = table[c - cStart] = [:]
    row.put("Channel", c)
    cFixed = op.ternary("transform.hyperSliceView").input(sdt, 3, (long) c).apply()
    for (t in (tStart..tEnd)) {
        sample = op.ternary("transform.hyperSliceView").input(cFixed, 2, (long) t).apply()
        row.put(String.format("%.1f ns", t * timeBase / timeBins), sample)
    }
}
ij.notebook().display(table)

[INFO] Reading SDT header
Dim #0: size: 128, type: X
Dim #1: size: 128, type: Y
Dim #2: size:  64, type: Lifetime
Dim #3: size:  16, type: Spectra
Time base: 12.500000, number of bins: 64


Channel 1.0 ns 1.2 ns 1.4 ns 1.6 ns 1.8 ns 2.0 ns 2.1 ns 2.3 ns 2.5 ns 2.7 ns 2.9 ns 3.1 ns 6 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAJDElEQVR4Xu2cjXLbOAyE85J9yb7k3ai1Umax+CNBipJ9M9/EXCxAEJRTJ+3c19fX13+V/Pr1S2gfNua4sLtc2speV+2zAc857IxzYE1cMyKeHXj1KQMrucuwHowQPkwm8tBHPEUI4e1YOOwdEcI3uwxmlz5OdutnECFM52EDnMaiOS3b6BZkZ5H19+ZMRAgfJrLZ5R8I4TJmDGdGzYchBEE7xKsHevX+T+I1y78vfv/+LQxXUn3R1fUOZtS8ACGU8ZABPR0hLOVJD8l5lpudSQhlXD2InT67jGCdA9cdCKGUbINZ/wdJcoZCWEay0VtQcaaKGgmE8EEhezFZf4bC2kJ4BIUDejpC2A7rMntjq9ihBwchbM0NBno3/r544mCzZ9L8mn5nmjMJ4YPDrrPq7EsIZXQ2NIWVvazcqwAhPIoVl1G1R1WdJEIY4qJDDDPS90huZQ0kWFMIlxFs+K2ZMCMhlDCh0TQ79HADhDDMnQY/s9eZtU8K9hDCdFjTp8ZilWTqZ7xXMtinELZn1cPyZJrZyeBurL7olfut3EtBCJSeRjM5rTeT15uT9V8F9tn+412MdfJvUVSwiyv2Xv3Q9eZMRggfFKouz6pjxUZRagthCqPvtiqie0d9vf4RrL2smIIQtqPjUB/iCCHFEy5nxRlW7NHFto0Vc8U5r9izAyG4zDjYLp8RnsY5S2OmQnAxim3PnXufhBBS9A408GS+BRucXwiCkSZHcivI7p/1Z5hZewAhbEV2aFn/2/PkgUXOFvEwsnme34t7DOQLQTBQfCq79nUzhLAV7SVrF67pq4n0EfEsRghLmTkQrbama5x+zMP1TRHCY3jIBakUnU8IpRQ1meaqfS2yPWX9jLaGUk9P2BGtP03vITC05UzsQwjfVG4aqRXxzKanh54cj0zNjJdgF2KapUepeJedeb35FVh7W7EI0fyoT/FLwygVl+sRqYsefGC8+K4c/RX2KASX0c1ZPvvXrsy3E7P6rKwbqCGEIbwNs/F2jTFNW81ID1f8P5qhX2lAMx4Q1yNU1jqorqfB5nLq7DWDeb2cDMFaQsgkmznRQ3lxJOJnHqadOrsMJKtbRHPO7xCe34sbXmkwzH+Y/W2L7enFTl2Loy9DtLbFSO4sXj3JwAgjBx3JjeajB9eraPc9Xmf6OLzWmzBSr4nLIBgoZxwPgj4k4vFgNaxDazqDebE2O7sG5jK8uAX76SlC4xUCBeN4MIxHPVmsGhjDdTTGOP1WnhXz4hjDtUfW3yD/TLeKWbEPPvimODjmj3cwm6YHGVSMKbS84z/Uqjj3jH5yRrL+qtwRzn1xf1xj7DtuGRlZ/wzYuwjjqEXeYSwvw2h+S2WtFlJXmjRI8nRW7IkP1PGw9OyLOee61dHD/BoY99ZB4gXaQXle5sE149wj4sW89iuLeVoP1r4zOfdjf9wl5ycEgVfMi/eCh7LiqLHYTnhnY1T7XgihjPNJZA1ZOmq9sFqo4RpjrE+m9ZKtw/xMs4DPQ/2FmJ9pHl6OF/eIXpjm0XRGxV5nTPNoRP3go6Kr4Vd83UM23xoWrluivz3TajNPFawe0zJxRjMDGZxJpNnIj2wI1sV1FMzDNWoY12Lo03LY2a3cAoQQgjUVPXCESP7pQa+mn9oJxjAHPbhmeB4vjkT9EZ9yNmnMgBvjehZsH6ZZeob2odHqRXWtDluj1ktbB2pKc4SqxlZgHP4b9q1XY9bFRPRRSN2fBjYIkrQcrQfUca0R9fUwUtvL9eKWh93tFxFc2AZRDTk8Ed/pbb9WEq2JPlwjXhw96Md1hGjOyycTcI1ocesgmg9hsag2ylFTq6vpFmcOy2VaBuXd/E24fthYxDlktu+IFkHLs3QWazXrkjUyXouiOn9feE/UCGxgUaL+iC/iiZKpFfVaPi2m6R5Nngz20NNIJge9uEbOuOdjsJxDQx3XGujz1vhmxLgWs3wq7K8Te7DyrRjjHHY27wp6e4zkaXPAtYfzkMgEBkmkzWneCL15WaJ/F3DGNY+mW3F8d2dhNRln3wG/EARnEa0Y05m2mmgPeCnRPMzJ5HneNt7Wxq8WnucVlwHEK9RLdmiYi5rGLG+Pn+XgZXv+Xl2JS8MqvEZ3J9s/+nFdQcfDJItUwBrBzXHNwG/PmVzkyMG+eupkwT1wrYE+XGOMzdnKefHTiF+znI1o+dgc+nCNuhbXfJ6fwXKYFqE9o1VDe9C1vLYmi6PPQAhusqZrzPZrsDpMy2DlazH8qQN9uEZdi6OvEyH8gD2ZIxtGDxUlUocNvYe2Rk89LUfTvbimazGmfRGhC6U45fCyB+sKMn1rRB8MK6Z5cI148QBCGMJqiA0q4rc8FX7GGWt/U4p+bz+mM01jlheIFbFiFXEPlh/VND1yqRi3yHijsJqtxuIeP3LaxS7flg+0Q2qvLc3SZ3iiWLUwxs5teRII4Q+RYhFPFdG9Ir6IJwK7FFyj7mHVRI44ejr2FUII3ADXFp7XiyPRQ1vx7OAtbaQfplVg1BVCCHbgLEeel+vFZ5DZk50B1xaaN6tHUHKF4KIUcmOM2f5MTtSHHHn4yx6MYw4DfYs+k/1bYAO4juLlefEqrH2OGIu3GotXs2IPByEMUXmg3lqZvIyXkclnXu1BzID5uHYQQgiv8Z53klfzLlSf4axXXfeFEP7gbaZdsJVnxTJk61T4d3s4WT+4DiKEFJ2bXk5v35G883JObyQH81EbgdVrNJngwQpG6B2IBavFNIvWH82N+iqYupdX3IpbsQxVdSrYqZdFCGHKEEZrevlePMIuNRYjhG+qD7OynhX78AMhfIZ3AWzmTIvEkgjhMRQOqYTd+nkhhCW0wxgdjJfvxe/AxDMI4ZZMHNBS2DmYZpH0C2ELkoe4HSPn03I13UEIw3Q20kXFXl4NL56hspZHcC8hTCHYTJjeem1eb42VaD1qegelxW7HcfZ3Pv8XCrsMY3YfI/VHcjdECB8Wc/EDJYTt0Qam6R9MhPBY7vYBcBH/FtVDidaL+p7GJucWwqVoQ9H0D8MIgTL7Ao76s/e4mk3PJ4THYl2AFXs4QkjzxsN7AkIo510ekJ3OmehFCB/eCyF8eCP+By/+1tf1sKBUAAAAAElFTkSuQmCC" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAQ20lEQVR4XtWbDXLkthKD52g5Wo6Wm71Xcty78Ef0DymN10kVMiKARjcpzXi82bxer9f/Kvz111/2uuIm2i4i68nMU9yZ4U7tm7AQH5gO2vkufeIh12kZ/y58Z7/v7PUyxC/w5k0Gm3ju+J8E9zfB7nk8gaf7IG810HxngKw24zPQP7kRGV/hpOZO7UnNk/Wv618PhGyh69fpHbp66lxnXMWr/vfffy98hS6zwp3aD9wOKKDZ0z7hm/pdrUOlnfiI6V7j4aCH63fjV7+q8Tu0KU4yTmqewk7vHe834N+La6jJYPRwnWHqy3C33mXomlrGTTA5y07/NrhBHPcE7uSyluunMLl59JPLEN6dmqdheq8mB1M40ujbPeCfgnfNXOVmZ+W4iZZgIT6g32QPQi2mOVPfu3DSv6sJPbupRObJ+E4rsBC3cDjEqHZyiI533Dugc93t+Y31W+Ytr4Ord5zTKt9PQjdnpxO7/gkkcxWNacT/VzB9l3b6FFlOxk/w4BtjIcrASutw1Qao3YFmviN7wk3wrv2fQGZYxT+ByROd8U9j2qfynWoOu/4OyFsNO3DDveM3iMBTeZOc8Ey8E9zNu

Shown above are images from channel 6 through 15, time bin 5 through 16. For the rest of the demo, we choose channel 12 and perform the fit from time bin 9 to 20.

## Hyperparameter Setup

Prior to fitting, we set up some fitting parameters specifying how the fitting is done. All the settings are described below. The commented settings are optional and are set to default values.

In [6]:
import org.scijava.ops.flim.FitParams
// import flimlib.flimj.FitFunc
// import flimlib.flimj.NoiseType
// import flimlib.flimj.RestrainType

// create a new fitting parameter set
param = new FitParams()
// the dataset (3D image with coordinates (x, y, t)) we choose channel 12 in this case
param.transMap = op.ternary("transform.hyperSliceView").input(sdt, 3, (long) 12).apply();
// // the iterative fitting routine will stop when chi-squared improvement is less than param.chisq_delta
// param.chisq_delta = 0.0001f
// // the confidence interval when calculating the error axes (95% here)
// param.chisq_percent = 95
// // the routine will also stop when chi-squared < param.chisq_target
// param.chisq_target = 1
// when does the decay start and end?
// param.fitStart = 9
// param.fitEnd = 20
// // the deacy model to use, in this case y(t) = Z + A * e^(-t / TAU)
// param.fitFunc = FitFunc.GCI_MULTIEXP_TAU
// // assume the data noise follows a Poisson distribution
// param.noise = NoiseType.NOISE_GAUSSIAN_FIT
// // the standard deviation at each data point in y
// // NB: if NoiseType.NOISE_GIVEN is used, param.sig should be passed in
// param.sig = null
// // initial Z, A_i and TAU_i (i = 1, 2, ...)
// param.param = [ 0, 0, 0, ... ]
// all three parameters above will be fitted
// param.paramFree = [ true, true, true ]
// // use the default restrain type
// param.restrain = RestrainType.ECF_RESTRAIN_DEFAULT
// the time difference between two consecutive bins (ns)
param.xInc = timeBase / timeBins
// // generates the image of return code
// param.getReturnCodeMap = false
// // generates the image of parameters
// param.getParamMap = true
// // generates the image of fitted data
// param.getFittedMap = false
// // generates the image of residuals
// param.getResidualsMap = false
// // generates the image of chi-squared
// param.getChisqMap = false
// the index of the lifetime axis (from metadata)
param.ltAxis = 2

param

xInc: 0.195313, interval: [-1, -1), intensity threshold: 0.000000, instr: null, noise: NOISE_POISSON_FIT, sig: null, param: null, paramFree: null, restrain: ECF_RESTRAIN_DEFAULT, fitFunc: flimlib.FitFuncNative@30e9ac9e, chisq_target: 1.000000, chisq_delta: 0.000100, chisq_percent: 95

All of the fitting ops takes the same parameter, the fitting parameter (`params`) and the Lifetime axis index (`lifetimeAxis`). The rigion of interest (`roi`) is optional (see below).

In [7]:
op.helpVerbose("flim.fitLMA")

flim.fitLMA:
	- org.scijava.ops.flim.impl.LMAFit$LMASingleFitRAI
		> input1 : org.scijava.ops.flim.FitParams<I extends net.imglib2.type.numeric.RealType<I>>
		> input2 (optional) : net.imglib2.roi.RealMask
		> input3 (optional) : net.imglib2.RandomAccessibleInterval<K extends net.imglib2.type.numeric.RealType<K>>
		> input4 (optional) : org.scijava.ops.flim.FitWorker.FitEventHandler<I extends net.imglib2.type.numeric.RealType<I>>
		Returns : org.scijava.ops.flim.FitResults

## Performing Image Fitting

Once everything is set up, the fitting routine can be easily started. The op will generate an `FitResults` object with all the per-pixel results assembled into images. Specifically, `resutls.paramMap` will be the image of fitted parameters if `param.getParamMap` is set to `true` (which is by default), and `resutls.fittedMap`, `resutls.residualMap`, `resutls.chisqMap` will be those of fitted data ($\tilde{y}$), residuals ($y-\tilde{y}$) and $\chi^2$ respectively if the corresponding `getXxMap` option is turned on.<br>

This images (`xxMap`) in `results` will be of the same size as the input dataset in X and Y directions. The result attributes (fitted parameters, $\chi^2$, etc.) for that (x, y) coordinate will be layed along the Lifetime axis. E.g. `results.paramMap(x, y, 0)` will be the *Z* (constant term) for the transient at coordinate (x, y), while `results.fittedMap(x, y, 4)` will be the fitted data of the 4th time bin ($\tilde{y}_4$) of the same pixel.

Here we demonstrate the most used ops:

### Initial Parameter Estimation (RLD)

In [8]:
// spin!
rldRslt = op.unary("flim.fitRLD").input(param).apply()

org.scijava.ops.flim.FitResults@459b1492

In [9]:
// showing tau from 0.15 to 0.4
nb.display(fcd.tableDisp(rldRslt, 0.15, 0.4))

0.0
Z min = -99.99993896484375
Z max = 10.958125114440918
A1 min = 0.0
A1 max = 1728.772216796875
Tau1 min = 0.0
Tau1 max = 5.780399322509766
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAgRElEQVR4Xu3dx44kSxmG4YP33nvvvffee+9BQkJ4hF/gFogNCJBgxwIhQOxYsQJxA+y4AS6DGxj0pPSWYiKqxxzOVPdM1UifMvMP//uIzOq57bbbbrt0wlFjIZxwXFgIJxwXFsIJx4WFcMJxYSGccFxYCCccFxbCCceFhXDCcWEhnHBcWAgnHBcWwgnHhYVwwnFhIZxwXFgIJxwXFsIJx4WFcMJxYSHclHjAAx6w0E64JiyEWxKPf/zjd/dXU5bKr1bvFsFCOFfc//73X2jXioc85CG7+4c+9KGX7nGPe2z39773vXf0hPq85z1vu973vve99KAHPeiyfo5E8GEhHBQPe9jDFtoIgpxpozXvw7Of/eztSsiPe9zjtvv73e9+l+51r3tdutvd7rY9EzIYP6XzvE9ZRujvFvMQC+GGY2TyPjpBPeEJT7isjDUrvxLTlT/5yU/e7h/96Edfeu5zn7ujj4r0lre8ZdeXOve5z30uPepRj1r6g6sp2y2gDAvh4MjyZmsfFSIaYbHYqzFcX7Oiec4j1NfYjxAyh6DmJFSkULyJq+erzeMmwEI4GAhgvM/qMfWtb33rZXUTZi789qIx9ffwhz/8srJnPetZ29X46iXwRzziEdu1XKFn4UNdz8985jMXhbtJsBBuGB7zmMfs7mMuiNmsarYmzyzwbW9722KZPeunhA5YsXaNtU8os+BH5PJH5TSWfvRtPvp//vOfvxO+8feNc5NgIdxQxNgHPvCBOxrLHxnO0lIIzE/Yo+tXjumuY9tZicadwQx1x/qEm5VHL6EMb37zm3dleYrGzzPcZFgINwwYFNOyNMldysCSoou5rgSYdSX0+nP/xje+cRnnajAGpdM+AcoNKADL9vyc5zxnq5unch29kHpzvyNGpbzgWAgHw5zpZ5GYC2JyWbp4274eCJGQRgs8yw2fJQwCf/rTn77dCwvmAxStscZwob45meM41ux1qruPfgGxEO5wjActM1N6Zl2veMUrNi8weoiUAkNHF4tGUNqVlV8r5jnAXe961+2a55n7tBMZvdHYz76cojLrGHOfC4iFcIeAEMdY6Yq5JU+eCXS0zg5ZgNJgeoqQV6AgowCFj9nyc9Xoc5nn+fBp9iCsPNq4NW0HMm9Xn/SkJ11Gc08p9inaBcRCuEOQe+dKc6cYS+gEiUkYVyKnfGQ6IXpWt/YEgKn7XPq4Exgxxu1ZGYJdRvf6nxUEGnPejYwoL+gwajy/uMBYCP83RkG+5CUv2QSf9RAC5XjBC16wucb5pG08qFH3hS984aWXvexlWz8UYUweqzOPvw/mNLv1UMJXX+q6z4LblVSf15m9m6uzgHve856XjVlZfT71qU/d7sd1njMWwv+N2fWxDEwbBQiY2rP74u9ohe7vcpe7bPdZecx0n5VRtpE+Y57TWWhHUj8Jb8xjlHU2UMhCb5tqnRTcenp27Xke85yxEG4X7n73u18m3NGNll0XFoqXr33ta3d1YjBmx1RWoo2yP/7xj5e535mRT3va0y57HqGucLPPJTeevnvfgJ6HGRVwbDPnIp034MFnP/vZLQy4L4yV3wiBs9c7ZyyE/wsJzL2FYoSFu4eY5vrgBz94aR8wrViKiZ3w1daVIp1l8fM+vWPeGQQ5by/nOjAK272xP/nJT27jSPjGPEYfPedR0MthrjTOOWAh3C44MbPItmUEk+vH5IQJJXjK9sXlYiehFZ9HgWax4wnh2H4UVn3NuUP07hPqvj5meE/RGl/0ohft2o2K5Pwij5fip/RCwQU6NVwI14WEHgNnyyN4HiEhjUnS2Mf4rB/WgnFZuOscl+d+wlnCK4fQDwGkiHO9sQ9CVIcyFvvRX/e6111605vetAt149G2+sJNYYD7pxySRGcd5TqFh3nsA2MhXDdiVgciCaiTPEx47GMfuzH94x//+GVMzzJiBCE95SlP2cVPz6ys+rPACKXY3rlCfVYnCxVG8gR5EMo5n/aN/Xv2bYG+O5egFDwTJaUAlCEFoBQE3rrReMcUx/y0OSt0nQMWwnUjobTVS3gY03MuDyNYgPJXv/rVu9ygMrBVwqDXv/71uwOg0RtQpnkOCZ2ljWcCaB3KjEwv9JwliNY0vrUMn/nMZ3ZjUAYhIYXrEAndXNyHeQ5oZ41/QCyEa0ZJTRbcBxUWznKAEowZ8xgnOyIlaPeYhiHqExDB9apVv/qjHO7ngx91WGWW1nhnWZuxCkdnxePWpv0rX/nKbe7WnLcyHmsf56KvDIGiU1btrLFQqWzkwzljIVwXOqlzzWrcv+9977v0jGc8Y2MOS7cNU1Ys7aj3Va961cYcAteOkGMS6xuFl1L1XOI5zmeMxfsQ4+t3dP/GnRWoe4Itb4juBHEMIeaS8JsXz9WYY8JbYnilndCBsBCuGQTAkjBCgoNBX//617eYjQEW2V6fIihvi4hpWUX9ZdmsStloJeOW0JUijG1hnyVTqrGe/ile1o3mHf8osPkbAmOneNWjQPMOhsJbn7oUu4Oud73rXZc+9alPbfPrpZP7FG6fhzogFsI1weJioOuLX/ziXXbL7RIwJnzuc5/bHYxgDOZqy/0SThlyMXPM1DGbsIQHdZWNMdm4V7P45jM+E9RcZz4nGD1AyeX4kmiuF91c3d/5znfeQgSe/OpXv9rurRsfRt6N/YynjQfEQrgmjF/O5LpjEloewJWnsPAPfehDGzMJm3Jgzqc//emtbVagTQwq9lMCyjGfyM0Y23iehRVmz6GNuUR3qpii9aGI+9y79VJwa3HP+9WPOaj/5z//eatv7p69y9C/OfGY2r30pS/d2lDifa+UD4SFcEUkVPcE1fm2BbmK923/+lAS5ATveMc7tnYUIItrv+8eY8rwHS1jXMz1Kbe+E8YoxISHibfHnVpPFklQ0ef4rJ41C2spvHWPu52//OUv25ooPB5kKO1o6mv2SrNHOCAWwlWRpRFkE7e43LS9MBoGFJcJlhIQ4ug9uEx9cZGu+mhfrw0m6w+TE0h9qsd6XL/xjW9sdc0pBR2VROxN0cY1zCFkdsPqGI/ArYfi6sfceC/zUt7cgYKYg7WOCkJBv/3tb280CpJCmMud7nSnveMfAAvhiihBKnlhoRaTlY/JGsF5LjdowQSWa8VIDJUTsL59FjwKcvQ+2onno2WBPjC7hCuMOwh1xn4pl2fCGJNTY7DolO8Nb3jDNh5BJWgCdhagf/VTmL5X/NjHPrbxwbo7Mxi9jiS0NuN8D4SFcCayoE71TNjii22exWr13McgOUBhAjO118ZzAh+tlhUTMMY0dswaFWTfO/U8E8/CuvVLWNrnWQpX7s2T8pljllxfLP5b3/rW7kDHvCmpUPW1r31t66ecp5zBuMZhDMrM0fPb3/72jS/aqNsZgjb7di8HxEK4IghwtOhcFiblDjv8qL6POjAJk1MGjBpdOYvF5Nw4QcgD5vE/8YlPbO0xs3nM9V7zmtfs7sctXcIdQwHM2zkwn7a5o+ciVHPnJcD2zrY3pVfPGpR51vaRj3zk1k+vhNWFDsnKGbRlAHN+cIOxEM5EVmOSYwxz/8QnPnFbsGSNAHLtFodp4iHG5UpHRtSvdwAxcU7AAiHqb/5eP+uG3raxMow33zJ4wHjbtMYd+04B26dHp9BcuvkS7l//+tfti+J3vvOdO4XK5WuXd1THPOKVq7DpZFE/2uo/ZYa84YGwEM4Ea2pyo9uyWJ6AYCywMPDd7353c3dptwVnBaxmDAFoKU31Z6hb+wRqzNliuGnzMQ9hqJdAeSz9GEO5/pQl8GI/y3TvBNNY2hDaRz7ykU3J9aUPig3WNoYC6BBMe22M41zEWMpGxVGnV+djGDoAFsJeYHJWaYJea1bmmRJklSWILEkbzMoCxnNzDCMASsW

### Refinement (Levenberg-Marquardt Algorithm)

#### Plaint LMA fit

In [10]:
lmaRslt = op.unary("flim.fitLMA").input(param).apply()

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

0.0
Z min = -0.3008432686328888
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.395394802093506
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAxHUlEQVR4Xu3c1ZIkybWF4REdMTMzMzMzMzMzMzMzM1+I0UQmk5kupGfQC8h0oaeoY5/P/D3REZlZWdA13eoasz0R6eG419rgHlF9zjnnnLNzlHLRi150UbYfudCFLrQoO1VyWHM+TWVRcMbIFJhLXOISJz27zGUus6h/EJn3fxRy4QtfeFF2CmRRcIHJJqs+bECP5YQsCo5MNgF+LEcmi4Kt5Yhc1ElyTJpDl3NvNsW4/SZB//d//3fS/V4JM22/Ti5+8YsvyvYq+13f/4gsCjbKJqIctmwz1kUucpFF2WHIqSTFqex7H7IoODJZ584Pw6r3K6cTOHv1mPuURcG+5bAmfLGLXWxRtheZz2MbTzJvcxDZZrzTSBYFC1lnqcletmiXvexlF2Xbym7zOIgcluUfVj9HKIuCrWUT0y95yUsuypJVcXue8K1SZJ5hGiLmlls/c7KsI+mqcc4yWRTsWXZT4iaizGW3vg4qm+ay6RmZk+p/RE4u2E0Jc5lb7ipZZfFHLes8wLGcsz3ou1nnYVtI87rCFa4wruuSw3kYWNUHMf95+LjUpS514ve6+e+27t2eb5IrXvGKi7IjlkXBgYXS51u5gyhpv2IeQAV0Xmiem6zyTuuIcOlLX3pRlmxrRKehLAoOLHOwN1noXiXApsBNActbBPQ6r4Gg+nBtvvN5n0o5yrF2kUXB1rKXRQApcKayLoeYumZAieGr2hNby8td7nJjPle5ylVGmbpXutKVhtW6N86q+SKAsViwfrhk40Uc8+5e+2kfq/rbTU5DT7EoOBKZK6/fU5CBVigpVgYORV772tfeufzlLz+ADhyAX/Oa1xzteAHPeSB9ARhBrnOd64x66ii/+tWvPvpWVxtkMkbjqDslhTrrwsQZKIuCPcncxebuXecgk3nZ9CURQK5//esPoBCBoufh48pXvvKw2OIxYvAO5qEvzwEKMH0ACiFudrObjXJlxrjuda97gmDqIAYCqINQ7rUpfyARx/h5lU15wX7kAiDW0i1N4+uqJGk3WbWI+RiVAe6qV73qUKZ7APgNBAIYSmaVnrkSoCMKAN273v72tx+kudrVrrZzk5vcZOc2t7nNaHvve99754Y3vOEgx01vetNBmmtc4xqjfyDzJH4bX1/GAbyx/W5uxtM/AinXRp/W4vecrGeILApOuUxdOyVTKFAiQvGeglmqOlmbene84x0HCShcm1z1ve51r50b3OAGO7e4xS127nCHO+w85CEPGXUf+chH7tzlLnfZefjDHz6uPAILZ9Hkdre73RgLORACgQCrT/c3vvGNB4EQRntjG1d95YgRSeZrPQNkUbCQVV5gnetbZf3T9kCjSGXABTRFKwMcYgAc0J4DBqjulQEvKwUEd0yAcKMb3Wh4AddHPOIRO/e73/1G/w960IMGoDe/+c13rnWta+08+MEPHuCpd6c73WmMC3h11TGm/hDTM9db3vKWY/7mAWj9XO961xttkCCvMQ9xyWGQY5UXPQQ53FMyC11FmMQzLhfgrgBlTa6UCyhWRwCvvmeUTelTK2W9ES5i1CerVVe7QEUE96xf3/e85z3Hc20iG0IYB+hEH+ZqLNavLY+hn7vd7W6DREgg5Jg7UprPQXS6jkSnSBYFJ8l8IfuJc6y6eO030CgekBR861vfeliQmE2J5Fa3utUJolA+C1SHohEGwMDg6pFE/9p5llfQjjdRBnB1gEzBQgOgrQdo5nDXu9513CMD8D035t3vfvfRN+8SAUs2tXvMYx4z5qedvpXLFeaHYZtkk3VvenYIsig4iYGrXPoqWcfalIBIwPAbIKxIGLnzne883CwrAjjFA51b9RtwhIJZHcsXClgekmgPFPPURhlQgKGfkkWg8AzqGEPMplh9IZI6yHWf+9xntEUeZEEKZDE+4COffoQY41mLOehTGySy1oBb5f53M6Tdnh+iLArWyrZkINMcATlYZNYISApEBkq6xz3uMazLb2AACMAUTgCqPrCRQt+sk/VL/CR5ytq7l+QZD3kiH3AQjfWqD2TjuAceD2QcAOsfOYxZzNeXMfU1DUsIDPS2r9qp43lnEta5G6in2NLXyaJgIeuse9OEp4ulXC6RZVEgQFmO39wrkCmR9bEmcZg1sqqsX7wVg7VRRrnAk+n73R4/sIGGBH4D2hXwCHTb29529GMefhsbCbly42vval7GLvfIi6jvN1KrY408k/GNq9z8tetw6TR46bNOTi6gqHWAk03P5gJ4C2fdLCR3S6msnKKLqyV8ZeZAKtnigpVr4x6QnrPW+973vgNMVkb5JWH6Mv7jH//4AYayzhaQz3gsHgEBpw9inIc97GHj7KAQo715qG98HsJc2jXoEzmIOSEJb4NcvEL5z6bkeDdZFUYORfbi1qeyyZ0BQb8ApxQWCAwKpTwKIiVM3LhygFN2VgkMikSUBzzgASessfcKlC2T5xl4hKmrp3TzYH3KKJ+r1z/Rt3DEcpFSrNfOWYG25tShExFmeCHjaFPOAHzPrU+5Ps1NPePyEMjmGRLM30aSVWVHJnux6E1icfXlClCWqhzYLIc1ARlwLI9FAZAX8DxrznqUI4IyXgFIwC6GU7BxHvrQhw4geIWyd6KPMvticwlc8zYWYiIBQBEyQunLfBFLmRBV6AKaEOgK5IjdVlS/+kJa8/XbeIgz1525zcuOUBYFW0nubEog9xbYKRkFEiC7UiLQxV6KecYznjEUR3iJrAfwLAaoDnSe8pSnDECAqV/KBGbn8sYt4fJbO4ACCEnaCQDevTGmIakdCUst1AAasObsKkcoqdROHyV/6us/D8d7mI8Dp3YN5mYMc920O7gAZFFwQqbhYb6nLQRMryVcXKV72yRgSvxYD4WxfAp/2tOeNsrUBzZlFIMpimWpJzGjSPWRhycAMG9CkZRPuayOG84ls7qOl5GVR2jnoY66wDQW4FyBaj48AZKJ7/pFAsTr8Elf2nTu0KGWerxYZEV8a2rLiJzWoL22dGYec70fsSwKTpLdWBqbYzelUWDxEDgsIusv26cYbh9BKIGC3bOWQgDiKKNAV0onndixSIRixXIEc8md5mr17R5IniFDuwNrM455t1Mwvw6NzF15ZwcRyHNzePSjHz3KClPKzQMJeC0ktR6/Edtc1dencTfpdr+52Z5lVTI3z1bVmZcRjC67pSwxGAGARUEpiiVz+ayCIiOJ2J07ZRUAkH13MkjBgPb75S9/+SBErlu/rAh47R6QELhAN7f274hobHPgWZSZv/EQSlsgel54ACRQzcPzjo+Na+7mVt/Wy+uxcmR/29veNnYStqhtGa2r4+Tyg21BXoXRIcpya7eJmUnuqyxZGQVSqIVaHKUDTRkFUQ6FWXwHKPppjwxAoOiPoniIzu87z2f1WWdJVjmBcQGlfSHBM6QDQnO1vg50WGh7/I6MXYm5KNen+fEiyo0jLAGYu0cA87v//e8/1qlPZQTwiKRex8g8A2+ov21JsK3MQ/WusukwZ5NQjivyUDQrf+xjHzu8AMBdKYj7e8ITnjAUEaiUMd0zE5bXp1l+A/mJT3ziaIdIufwOYlhh7XP7gHVPyWXq6uizhA3wyhFDW3PXF5IgW2O3nfTcXD1viwo8a1QuzgMdmFy8e/WAbN68B0K09TS+9vTDIObGd+TSBPbKREqlqNwgZQEY6EDgti3wgQ984M6TnvSkYaGsBDCUyYooreyfgtwjASWxMKCxIlbeToKX0A6RgAQE+YZ74h6I5mgexJisWCbvmavxlOsbGOagvf5ZEeLQTdvO3vyZk3mUnAJTH8DnPYCtDsIyCnqQ8yCyPp785CcPozAv9elkaoTmMNd1EkaFhL1ilpR4nmfE6z/MXCWdaRNKpSRWYTFAokxAAg4BCIU//el

The plaint LMA fit starts with an arbitrary set of initial parameters $z=0, a_i=1, \tau_i=1$. By design, the algorithm is only able to find values that locally minimizes the residuals and is therefore harder to converge compared to the following scheme:

#### LMA fit with estimated initial values

To set the initial parameters, either use `param.param` to set an array of global initial values:

In [11]:
// Z = 0, A = 1000, Tau = 0.187
param.param = [ 0, 1000, 0.18723493814468384 ]
lmaRslt = op.unary("flim.fitLMA").input(param).apply()
// later fits shouldn't be affected
param.param = null

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

0.0
Z min = -81.83920288085938
Z max = 180.95030212402344
A1 min = -179.47975158691406
A1 max = 1088.8184814453125
Tau1 min = -1.5415652646912E13
Tau1 max = 7.7091735513491046E17
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAOAklEQVR4Xu2a247rRg5Fc89/5jX//5wBZ7CDNYssWfZpy3ZLATaqavNOlmS3c3766aef/nln/PXXX4278KVoxIVzoREXzoVGXDgXGnHhXGjEhXOhERfeDH///XfjvhCNuHAuNOLCudCIC+dCIy6cC434Fqj/zF0Y0YgL50IjLpwLjXg5fv7558ZdeBoaceFcaMSFc6ERF86FRnxLPPK94hGbD0QjPhYcWO1zNr+yOSka8fZYDY0DL/zyyy/jestuJd/afzD+t3mHf3z5ow3NAD3saZ1AHfqc+G+ERrw1OKwtTEO17eTH3KQfP5PsA9GIw+BGbjXUOl7vhe08UMuo4zzNfRgacQjc1JWc51XjJ97+9mCyneJwtewD0Yinw82aGjg1d6VTn/eOsRf0OcWZYm75+EA04odw68vkNNQtWMe+Hhk+/WaNL3PWpcz8FOMD0Igvh5uxaibPE2hrfoVJxz4s2wP7Wp0/AI14Gtw885Rb75at+RUch3yteaNYPv2G4POEPTovRiO+BC7cjefZMttY3zqroU2cz+S8mpviU2bevt4UjfgS3GpiIU+Vm8Rz6XDA1v0K7Ml1imvOul7fFI34YWw1MZheqbTN3j/jPqOp9uUYjGsdcysb+3wjNOKH4MLJEda1zJzlq/VHcY8fX8rYFyhz7m+GRjyM1TDcADZp8hE9NvHXX39tfm1nzvJJh7lRvtrnzNzto3LNR5d929cboBEPY2qiYX372Gu7x8eWXs5bvrdQNv5usgJtJv7FaMRDSENY4PT57YbYD7FlZ865rLh7ZSu9rf0Kk45jvACN+D/ck2TpTk8FOdvQlvsJK/29iA39Tb63bHL25Zz0JsTO/l+IRjyEW43wxZh0wuXzk2fGcWzncQ9nGc/Wcb4EbSZMMuf0IjTiYVRRHB6fksizRjdf7ojp6Zr8eM+Yzsu53pI5LjHlN8W3DnnKHftgNOIuuEBj9fqPjBfAPnOZ7HPyt+K2sFd/yiW8fVA25W79N0Aj7oKLc7NqwBwym7JqUJrkc7hpvxeP2MTOts458GXxxYmtY7wIjdgNF5jCOfDs3QQ3aULJHcvxp3XSMUfeOj5PeSQ/+7Gs8Ntvv/1b7+SLsV6ARuyCE/eNny4DG0Mdyo3Eop5zuReTjy3fU/wpx+hlyKzRdU55+HwQGrELLIaFs1gXzl/IfDFYvPfxtbdB0durb9+JZz3n69xjl/rcC/pgzL15PgmN2AU3oOCiCQ49+/JjfTfGvpzDlJe5W6Bvr5Oe9TPcqRbnT9s3QSN2IQW6WA6Q8nwZnJoycYlBHTbOerlcOU/5rvhptY7r4zr1YbKh3SqfF6ARN8FmsdgMOLKpYHPT3vEcm+Dgnd+EyPboZB/wzL9ufPkKt+pexXsBGnETLMgD5DkXwnI3p3xOXGyyZ2zrOjfaGCt+sndeGXyhvt2HS54F11j7XJIVnMeBaMQmnHgKTdFTodTxpYjPyXd4Io2OD8uNScfcFMt5BKmTtbge94TyLazyezIacRNOmoUa0ePgJ1/eG27WSi+6brrlE08Zc805XNXy559//vP777//+xaIzpTD5Hc6R885PRmN2ASTD5citoorTG8H+nYzDMq8EnwynVtysNywXfZbvp3X5Ju2rIu6rufJaMQSTND7qVjqUYf+7Ju2xiT3FzDGyMDyS1zxvITkPJStHKNffnmhmBvXFRKTnOMfgEZswslu7T0M+uA6+XVM6wdpYobNAWdQ4aenN3mSS668XOGcI304R/vnmX4mG9f5RDRiE0nQyZvnUKaiireddRw3NhNin5huOuXTwPl7/cpmipe8qFtr9OjP/iOPP9d0EBqxCTeouBQXnZz5BLFAclPTDA5qpRe5m+y4PidPXho/+dSxD8ajzykOcyJHmes6AI1Ygkm6QDeFhU5+fJ50yRfKL2U8e/B8/deaz2t+a48dL4AHHb/0zbjMk7m4FvL2lf1kcwAasQkWxz0LmPYukMWzCZE57gQOJ6gB159nGSI5x2PcXJD4cd6E82WdJeNl2uqHa7XPg9CIJfYmNzXLPqhDvcLqCeSrOXoZcHS4Lz3H54D55PMcjjlHJ7zzj2/GYKypN9P5BWjETaQogo3Juiou+is5/djGA+BTW/AlsQ+iLkue/JL7QtCGw5zOW3xydi7mE/NgNGIJF0uea/ZpRAbD5rIBbob9MF50J9Rrnq/6fN6XXXF5OzgPvnFWeQbMhznTJivzj96EW/InoxG7cCtpNipnNohNN9xovpIj9yDN1conPOAliO9cmtj6TP8Bc2W91qHuhKmXPj8ZjdgFFpczE3ehRJq7ugRpvJvIuBx0cRl2vgDStuz43YD2hchKj7nVvvw5P/pJHPLWZw0Tpv4diEaMmJIjxwLSPMtYrBuSQXI4vCD2xeEVn338Us5h+q2Qff3PncQMl4+Q5MHcE8eXLXvXTT75Uk4Z9wegEbuxKjbNta4bxXNWNpQyngscYs6FP/744796HGJ85O0QGf2mhvhLLow55WGZL8s9cH8PQiN2YUq6zhxmgbrRpx7ltM2aIdOHmx+UXp7ynGuf1zgvTPZ5yvm0J378VI6Jkfjkwju/qT7qTr15ARqxC0l4VXxkfKJYNO2iR7C5jmEftp9e8X5jxFe+8PHSRJ69a2EOxbkvgeuI3VSPseKfgEbcBJPLADgMYmp67NI86k260SdHHdsXMlAOtj4ezGdPcHCJFSSe+xF+6oF7NNXj84FoxC6kGBbOYlJwBuICaVOgjhvIxrGhPCdODZkXKX6ZF1/3tM3wmUNyoixILxKLdrxEtGEuPrO/Pj8RjbgJJsdCUnihZGl2njbqlZyDiq80Lb74RPKptT/KOVDa1+rvCM7dvPOiTvSSy5RX+kC9LbjXB6ARS0xJugDqZJ+msUFsaPTDZWgT3Pz455940TFXunn6g+gWz18RWWNiRY++Sp4Y8ec1+/hlfMd6ARqxCSfM4ggOl82IjwyRfjjc7MMX2HxeCsbLkPjkJ2f75OUgok+/kbmm5JIzV+/tO6u5g9GI3WATWKA5D5J68WNdDpOyaRgeatbY18pXf2zsL7LKyXEsD1Knc3F9tGOPDPf4ADTiLqQBKSB7rpM8xYZzk3kBpkFRzqc9fPxm6JH7TF3mxNyZF/UK9BX/zjU2BnvI88FoxN1gs9ikNCcNoYywbRBbohpcehw0ZZP/yJiLbW2T7wNE8a6dvkvHb6Apn8C+XoRG7EIKcFEFNqXOaUKdYz81xhchzQzytNM3YxYyuMTYisOhUX+6MGU31c78HIN6weTD3MFoxC6wKBbshrAxZefGuMmxCUdf0WWc2GRofsUXeJEmv66HOs7NiMw+uca/V3MvQiN2g4mzCW6SwWalyRxS6UzDpu/IIo8+bfwUM050nH/J6dNxS3fKkzpEeMdxD+n/YDRiN1bFspjwbnrtp6eR+mwu7Sc9fvZGxxcoeuQtTzx+1ORSMC4vFXN0vgT7wf55fzAacRNMOqsbxAakMdxHTp+2T3NXtivdyP1xwOEwrnVoR5/MJbknVvw6L/fMfZz4g9GIu8DBsXg2wY2xPhsRff4qV2DjHYuD9Su+VnKORVme+PjMvx8wXE9A3+mN++TeuZ8vQCPuggtzY9h0D4a2tOFAqGdbfsbXmUOKTu3jh3lnyBw687OMMaIXPx7+qr5pfQM0YjemIlj4hKlh5PmUWZc2HEytxW35JRfew/dF4GXKJUos+o4tc4yP6LtHW+eD0Yi7wcJ9djNY7EruxkbXcn5RK56XoWC/jBt7xko8c9G1LD6ZU2piLK5viEY8BBaefcDGuEmW5Zw1T2H8Fp9B1FD4r4CzMg/7z2VJLMbLhaD/nAn6TI70ldju0SuwI49G3I2paDbIKDkbz6YHWzrUzVAyCOb

or use `param.paramMap` to set the per-pixel initial values from a previous fit:

In [12]:
// here we use the RLD's output as our estimation
param.paramMap = rldRslt.paramMap
lmaRslt = op.unary("flim.fitLMA").input(param).apply()
// later fits shouldn't be affected
param.paramMap = null

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

0.0
Z min = -0.3008432686328888
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.395394802093506
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAxHUlEQVR4Xu3c1ZIkybWF4REdMTMzMzMzMzMzMzMzM1+I0UQmk5kupGfQC8h0oaeoY5/P/D3REZlZWdA13eoasz0R6eG419rgHlF9zjnnnLNzlHLRi150UbYfudCFLrQoO1VyWHM+TWVRcMbIFJhLXOISJz27zGUus6h/EJn3fxRy4QtfeFF2CmRRcIHJJqs+bECP5YQsCo5MNgF+LEcmi4Kt5Yhc1ElyTJpDl3NvNsW4/SZB//d//3fS/V4JM22/Ti5+8YsvyvYq+13f/4gsCjbKJqIctmwz1kUucpFF2WHIqSTFqex7H7IoODJZ584Pw6r3K6cTOHv1mPuURcG+5bAmfLGLXWxRtheZz2MbTzJvcxDZZrzTSBYFC1lnqcletmiXvexlF2Xbym7zOIgcluUfVj9HKIuCrWUT0y95yUsuypJVcXue8K1SZJ5hGiLmlls/c7KsI+mqcc4yWRTsWXZT4iaizGW3vg4qm+ay6RmZk+p/RE4u2E0Jc5lb7ipZZfFHLes8wLGcsz3ou1nnYVtI87rCFa4wruuSw3kYWNUHMf95+LjUpS514ve6+e+27t2eb5IrXvGKi7IjlkXBgYXS51u5gyhpv2IeQAV0Xmiem6zyTuuIcOlLX3pRlmxrRKehLAoOLHOwN1noXiXApsBNActbBPQ6r4Gg+nBtvvN5n0o5yrF2kUXB1rKXRQApcKayLoeYumZAieGr2hNby8td7nJjPle5ylVGmbpXutKVhtW6N86q+SKAsViwfrhk40Uc8+5e+2kfq/rbTU5DT7EoOBKZK6/fU5CBVigpVgYORV772tfeufzlLz+ADhyAX/Oa1xzteAHPeSB9ARhBrnOd64x66ii/+tWvPvpWVxtkMkbjqDslhTrrwsQZKIuCPcncxebuXecgk3nZ9CURQK5//esPoBCBoufh48pXvvKw2OIxYvAO5qEvzwEKMH0ACiFudrObjXJlxrjuda97gmDqIAYCqINQ7rUpfyARx/h5lU15wX7kAiDW0i1N4+uqJGk3WbWI+RiVAe6qV73qUKZ7APgNBAIYSmaVnrkSoCMKAN273v72tx+kudrVrrZzk5vcZOc2t7nNaHvve99754Y3vOEgx01vetNBmmtc4xqjfyDzJH4bX1/GAbyx/W5uxtM/AinXRp/W4vecrGeILApOuUxdOyVTKFAiQvGeglmqOlmbene84x0HCShcm1z1ve51r50b3OAGO7e4xS127nCHO+w85CEPGXUf+chH7tzlLnfZefjDHz6uPAILZ9Hkdre73RgLORACgQCrT/c3vvGNB4EQRntjG1d95YgRSeZrPQNkUbCQVV5gnetbZf3T9kCjSGXABTRFKwMcYgAc0J4DBqjulQEvKwUEd0yAcKMb3Wh4AddHPOIRO/e73/1G/w960IMGoDe/+c13rnWta+08+MEPHuCpd6c73WmMC3h11TGm/hDTM9db3vKWY/7mAWj9XO961xttkCCvMQ9xyWGQY5UXPQQ53FMyC11FmMQzLhfgrgBlTa6UCyhWRwCvvmeUTelTK2W9ES5i1CerVVe7QEUE96xf3/e85z3Hc20iG0IYB+hEH+ZqLNavLY+hn7vd7W6DREgg5Jg7UprPQXS6jkSnSBYFJ8l8IfuJc6y6eO030CgekBR861vfeliQmE2J5Fa3utUJolA+C1SHohEGwMDg6pFE/9p5llfQjjdRBnB1gEzBQgOgrQdo5nDXu9513CMD8D035t3vfvfRN+8SAUs2tXvMYx4z5qedvpXLFeaHYZtkk3VvenYIsig4iYGrXPoqWcfalIBIwPAbIKxIGLnzne883CwrAjjFA51b9RtwhIJZHcsXClgekmgPFPPURhlQgKGfkkWg8AzqGEPMplh9IZI6yHWf+9xntEUeZEEKZDE+4COffoQY41mLOehTGySy1oBb5f53M6Tdnh+iLArWyrZkINMcATlYZNYISApEBkq6xz3uMazLb2AACMAUTgCqPrCRQt+sk/VL/CR5ytq7l+QZD3kiH3AQjfWqD2TjuAceD2QcAOsfOYxZzNeXMfU1DUsIDPS2r9qp43lnEta5G6in2NLXyaJgIeuse9OEp4ulXC6RZVEgQFmO39wrkCmR9bEmcZg1sqqsX7wVg7VRRrnAk+n73R4/sIGGBH4D2hXwCHTb29529GMefhsbCbly42vval7GLvfIi6jvN1KrY408k/GNq9z8tetw6TR46bNOTi6gqHWAk03P5gJ4C2fdLCR3S6msnKKLqyV8ZeZAKtnigpVr4x6QnrPW+973vgNMVkb5JWH6Mv7jH//4AYayzhaQz3gsHgEBpw9inIc97GHj7KAQo715qG98HsJc2jXoEzmIOSEJb4NcvEL5z6bkeDdZFUYORfbi1qeyyZ0BQb8ApxQWCAwKpTwKIiVM3LhygFN2VgkMikSUBzzgASessfcKlC2T5xl4hKmrp3TzYH3KKJ+r1z/Rt3DEcpFSrNfOWYG25tShExFmeCHjaFPOAHzPrU+5Ps1NPePyEMjmGRLM30aSVWVHJnux6E1icfXlClCWqhzYLIc1ARlwLI9FAZAX8DxrznqUI4IyXgFIwC6GU7BxHvrQhw4geIWyd6KPMvticwlc8zYWYiIBQBEyQunLfBFLmRBV6AKaEOgK5IjdVlS/+kJa8/XbeIgz1525zcuOUBYFW0nubEog9xbYKRkFEiC7UiLQxV6KecYznjEUR3iJrAfwLAaoDnSe8pSnDECAqV/KBGbn8sYt4fJbO4ACCEnaCQDevTGmIakdCUst1AAasObsKkcoqdROHyV/6us/D8d7mI8Dp3YN5mYMc920O7gAZFFwQqbhYb6nLQRMryVcXKV72yRgSvxYD4WxfAp/2tOeNsrUBzZlFIMpimWpJzGjSPWRhycAMG9CkZRPuayOG84ls7qOl5GVR2jnoY66wDQW4FyBaj48AZKJ7/pFAsTr8Elf2nTu0KGWerxYZEV8a2rLiJzWoL22dGYec70fsSwKTpLdWBqbYzelUWDxEDgsIusv26cYbh9BKIGC3bOWQgDiKKNAV0onndixSIRixXIEc8md5mr17R5IniFDuwNrM455t1Mwvw6NzF15ZwcRyHNzePSjHz3KClPKzQMJeC0ktR6/Edtc1dencTfpdr+52Z5lVTI3z1bVmZcRjC67pSwxGAGARUEpiiVz+ayCIiOJ2J07ZRUAkH13MkjBgPb75S9/+SBErlu/rAh47R6QELhAN7f274hobHPgWZSZv/EQSlsgel54ACRQzcPzjo+Na+7mVt/Wy+uxcmR/29veNnYStqhtGa2r4+Tyg21BXoXRIcpya7eJmUnuqyxZGQVSqIVaHKUDTRkFUQ6FWXwHKPppjwxAoOiPoniIzu87z2f1WWdJVjmBcQGlfSHBM6QDQnO1vg50WGh7/I6MXYm5KNen+fEiyo0jLAGYu0cA87v//e8/1qlPZQTwiKRex8g8A2+ov21JsK3MQ/WusukwZ5NQjivyUDQrf+xjHzu8AMBdKYj7e8ITnjAUEaiUMd0zE5bXp1l+A/mJT3ziaIdIufwOYlhh7XP7gHVPyWXq6uizhA3wyhFDW3PXF5IgW2O3nfTcXD1viwo8a1QuzgMdmFy8e/WAbN68B0K09TS+9vTDIObGd+TSBPbKREqlqNwgZQEY6EDgti3wgQ984M6TnvSkYaGsBDCUyYooreyfgtwjASWxMKCxIlbeToKX0A6RgAQE+YZ74h6I5mgexJisWCbvmavxlOsbGOagvf5ZEeLQTdvO3vyZk3mUnAJTH8DnPYCtDsIyCnqQ8yCyPp785CcPozAv9elkaoTmMNd1EkaFhL1ilpR4nmfE6z/MXCWdaRNKpSRWYTFAokxAAg4BCIU//el

_Note: if both initial value settings are present, the global values will be overriden by the pixel-specific values._

### Global Analysis

In [13]:
globalRslt = op.unary("flim.fitGlobal").input(param).apply()

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(globalRslt, 0.13, 0.25))

0.0
Z min = -4.0
Z max = 5.352066993713379
A1 min = 0.0
A1 max = 1058.4622802734375
Tau1 min = 0.0
Tau1 max = 0.18723493814468384
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA6OUlEQVR4Xu3dc7QtSdIF8B7btm3btm3btm3btm3bto01M2vN/Fvf+mX3vp03TtbBfff16/mme63oqsrKzMqM2LEjMqvOffvtt99+07pyqEMdaqFsmRz60IeeLat95bqWV+n77Ouualfb1mvtI7VdL+vUIfVZq2TdfveCLBTMyjIjzRlj7nwdqUqZa9/Xq8dar5aNpNZjzFpWpY51rq9Vss58d1kOvKgPHE2qXldZB/nVYP15f+zZIuej/uuY6vWeyiZGWQWWTfo6iGShYEF2OujDHe5ws216o/Z1RnVHUkFSZa58JKNnjsr2lezlsSwULBWDmTNWvd5peb0/98xR+7myvo9lsqzeqHxUthPZrX52IPufjAYwMsBIcq96XW3fX8/10cuIIWqbnnL7OnUsuT/qp78/KttJu3XuHUxkoWBLRoMfla2SXolzykyZYx/7a71Rmypz5f29uTqj++uOJ3VHffSyafleloWCbbLOoNapU2XUZpXiRl49J3N91PJNjLtJvWVymMMcZqFsJLvxrDVkoWAj6Qe5bMD1XjVmf3/u3pGOdKSV/bruZdTfSGo/q8r7e3PHUd2dlO1V6RXVS182uj9XlyyL3XP9jurVssMf/vALbfpnVc9K3dBy7a/21d+bY4Z6XcuNYa7OqP5Ilo1r12WdB9Q6ua7ltQ4lLvO+1On7irEOe9jDLtSPgfs2dSyjMakfw0TynIyx72vUz+i6lu227O3+m9SJrPvQUZtRWT3vy2odMgLMXP+13zkGIPYk+v5r24CgL+vvjdr8P5Htytx0krX+qJ8ocGTcnUp97rKyufn15aM+RqAY9bOObFJ/k7q7IItKGVSavdcbdU7Zo3ZzUvtzjGf3fdd+Q/M5759NlAkrpAekNstCQB86cq/WGd2rMldnrnyZ7KTNrNTJ1UlVWq2SetVw1evr/bl+qoy8MOV9n/04M6cYG/07Hv3oR1/Ynu4BUfuqzx3pae56Tub6WLf9rkuU5WLZIPp6yyT1ar992xEYap16HoM7r5m2cisExyMc4QjtGG9X1z1Hxnfu2N9Pf2l7xCMecRvw6nE0xp3InrbfJVkoWCkjhcwZsr/fZ/ZRcO2jN7KyeGiMlTp9e/cCAMY973nP245HO9rR2t6Be4x65CMfeZso1/YoRzlKq6sOEBDtHPUTwOi/B18F4kgv9brWqXVru3Xq7ZEs67jei/JznvtzHl37G/WVtlEuRTMO5bs+1rGOtWUQbZS5ZqwY6bjHPe50qlOdajrxiU88nfzkJ2/npz3taaeTnexk0wlPeMLp1Kc+9XT2s5+93TvNaU4zneIUp2j3T3nKUzbR/vjHP/50ghOcoN131E4ZcBz1qEfdNg7XYYs6rzlZpY9Vsqftl8iBF6OHjMqW3a8GrXV7gMSzeZj4TMmOAOCcoo93vOO1ozLtnDPCSU960mY89chFLnKRZtgznOEM05nPfObGAhe96EWny1zmMtOFL3zh6RKXuMR0+ctffrrGNa4xXfnKV54ueclLtuM1r3nN6dznPvd0pjOdaTrnOc85nec852nn2gMQQAALUBmLc/VPcpKTTMc+9rG3NqfqXJdJ1dm69/aSLBomN9cdTN82hq1tGbqvF2/nZec617malzLkcY5znKbo05/+9NPpTne6pnjK5okM78hDMYB27vFgRr7CFa4wnf/855/OdrazTVe/+tWnBzzgAdPtbne76U53utN0netcZ7r1rW89Xfe6153ueMc7Tne/+92ni1/84tPVrna16UpXulIDzVnPetbpMY95TOvjVre6VWONS1/60q3exS52seksZznLdOMb37iBR91znOMcbdzHOMYxtkJQnXeVXtdVf+vIJnXXkpHhqywDxaYTco/hGZMheRRl8rZQNWPyXgrnkTz9RCc60Ra1X+ACF2gA4YG8Pn3x7hve8IbTbW972+nOd77zdP/733961KMeNT30oQ9tALj3ve893fe+950udalLTXe7292mG93oRtPlLne56Ra3uEXr8wY3uEFjCka/4AUv2MByvetdr7EE9rjsZS/bWONa17pW6wN41Ff3jGc849Zqo8656mWZfqpsUncdGfS39OZC49Tp0V5zgLl+lDMWwzEuI6NusZsn8yjKDF3zPGUo15FnKueBzvWFLY55zGM25lDvnve85/TgBz94euYznzk98pGPnG5yk5s0r9XuQQ96ULt/y1veshn/5je/eWOFm970pm0sDAwE5zvf+ZqBgQ87AJM2xDMwyLWvfe12D0CuetWrNjCoD6hVH71eRvfWlWXgimza537+1ydjI+k7zURqWc4DjL5OJi5xYixxmpHRNIOLvRTOIBTqnFF47ZOe9KR2TsGYgjcqd476CfrVJ+pH1eowNuq//vWvP93rXvdqz1ImTIjxAMSIV7ziFRsYGU4Y8CxlGOEqV7nKdJvb3Ga6613v2oDE+7GM/rGAeze72c0aUxgf4JgPFutXPL2Oclym79puB0bdRPY/mRvQMnCkvE6u3ncUuyVyF7rQhVp8ZRDez/MIQPAk9O+IynkjhfLaeJgwILHThpF4ZJI0R8aUSMoRGFq/SQwZCHiATF+YRwgRfvRnTLw8YQgA5BCPeMQjmgCU3IHBPRsgsAHBJBgEeIQyQO8Tw5EO55xob0t51uKN3rCjgede31E8P9fZcaMEVG1ZRdGydwrigZTF2LyOou9whzs02g/V8zTK5WV3uctdWj2Gk6GL2+q7Fh6EAEaWODKOnEAZVuD1YrS6QKC9BFIYABLg8Txe7ZmeJ48AFDmD3EHOcJ/73Gd63OMe1+4Bwz3ucY82LmMBLKCRm5irZwt19hfmVgjLjL7s3i7Ldlpf58F9vbl2fTmPpBjeHPqkXHQqnlNePF45o2EHYHCtPgoGBu0ZWjkPZSCxWBiRjcsJ5BMYIPsIlnJEHW3VYSQAJJ4ldOiTJ3sG77797W+/Fe+VCyEYAPCEBfWFAYkk0AIVAApJyVuAPTuLFQhVZ6vK95IsGnFuALU81/H2iMnyehTrHtpPjGY0WbbYzLD3u9/9mlIf/vCHN6WiWtk7ZfN4xg0wHv/4x7cyFJ18gaBtyscunp1NIkfC6z0fCFG9tmhcKAEsXgyEPDk5hnJjUwYkGMKK4oEPfGADBsMbl3yFoc3NaoXHA7pwgnUAQn6BDQKAqscqe3p/Q9neYU/lowelrIaG/lodCZ9NGxTLgyjpaU972lacJag29M74PEscZVRLNzSrnLJRMMXrC2UDj7YBEsPaU8h7fwDIJg7vZADezzhCAoMyoGcClZWCkCDf8EwAcQ/le75nAwTGEa7UBxZgZHhgB0D9CzHu6U8/np+9i6rHKpuW71S6/hYKFq6drwMS53nJwvDol2cwqmSMJ/E+9Ep4l/hKmeIrz5cTYAlMwBiAwBBAYnmHkpMw6leZeoyEyikb+Dyb0iWfEjLXtoK1w0ASUqzAg4UhY/BczwE8RwYU7xkTQwGBcbqP9uUUKF68F36wUMKBpePLX/7yrXxBuPM8DNHrueq7P1bd1ja7IrXjuQFEquenXoTnUUjiO0/jcRImtM7zeTiKZXBKBQxHykPX2a61WnjKU54yvelNb2ptH/KQhzSDMTRvl8EDg76wgZVCdhB5O4NjHH1RvLiP3nlqxo6e1TFOz2NsQHCOWYwXIzzvec+bnvjEJzYm4tVCE1ABu5wjK4bnPOc5WyEGaADB8+UzwGxMVc990j3S+TLZtP5AlneUQY0SmNqGUVAhWqRIBqFIxmVAFMobXL/61a+e3va2t7XtWrTOsJI0cZmBUal4K2zYnrWWR79Za+sfABjbNe8TBiRglKwPIDAGnuy+RFMZVhAS8v4hrGNsnsloj33sYxvbYKKEKN5vfBglL630py1P11Z9YUM

### Multiple component fit

For multi-exponential models ($I=\sum_{i=1}^n a_i e^{-\frac{t}{\tau_i}}$), set `param.nComp` to the number of exponential components:

In [14]:
param.nComp = 2
// fitLMA automatically performs a RLF fit if no estimation is provided
// so the following step is redundant
// param.paramMap = rldRslt.paramMap
lmaRslt = op.unary("flim.fitLMA").input(param).apply()
// later fits shouldn't be affected
param.nComp = 1

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25, 900))

0.0
Z min = -71.99761962890625
Z max = 712.9345092773438
A1 min = -710.0867309570312
A1 max = 2269.291748046875
Tau1 min = -2.9590814988435456E16
Tau1 max = 4.596363691631163E24
A2 min = -1523.2769775390625
A2 max = 1083.8193359375
Tau2 min = -1.0956814942208E13
Tau2 max = 1.6905548549062656E16



Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAACVElEQVR4Xu3cS07rQBQA0TBAjLIN9r9DEOhFD3Xl6yTgdhXSEbhsxwq37TBit9vtPqKGEBeEuCDEBSEuu4+Xl5cxxgMhLghxQciTrewjFyEuCHFBiAtCXBDighAXhE16e3tDyzeEuCDEBSEuCHFBiAtCXBDighAXhLggxAUhLghxQYgLQlwQ4oIQF4S4IMQFIS4IcUHIH9rv92hPhpArfX2NbUIIU3h/f0fLIghxQYgLQlwQ4oIQF4S4IMQFIS4IcUGIC0KusLJ/9ngPhLggZAIPfAIhxAVhtR646vMfQlwQNq2nCCAoiRcGwtTEg1wKIS4I0xrv/nE7RyFsyrgIxu1jrjlmQxCm9DW0g3HfJUvO2RCEzfs58GPDv2cxTQhhCkuGs+QcAYSpjXfupaEf9o/fRRCmd2wBnBrspf0CCJt1bGEcfhYvBIQp3TLAa44RQbjbqbvrt5269rEmhrAZhwUwLsjxuHN9dM1rTQZBYyMDvBfCqtw6pPH4cTuAsCqXBjg+4kfn9uUbwqqdG+i5fc/0V9d9EITV+41f+G9cYyW282ZPvY9TPd8QNq3FAAhTOgy2Ad8MIS4Iq/DIO3npay09bzII+acFsHGSAV+CMI0G+BAIq/fzL/5nLoJnvvaKIMQFIS4IOs/+KFk5hLggKPUEiBVCXBDighAXhLgg3O319RUtq4UQF4S4IMQFIS4IcUGIC0JcEOKCEBeEuCDEBSEuCHFBiAtCXBDighAXhLggxAUhLghxQYgLQlwQ4oIQF4S4IMQFIS4IcUGIC0JcEOKCEJFPFanIdCeRf3AAAAAASUVORK5CYII=" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAyA0lEQVR4Xu3dB5RkVdUFYJIkEUWCCIIJBEkiiCASVVCSgiAZRFSSIEEFBEQUAZWoogRFJEkwkHMUBSWIZJAkBjCAJCWqPNd3/ff81XVrenpmurp7qKq17qqq+9674ex99jn3vlfdU0011VRNv/R0qSr6ZYyUqaeeuqrrQqkq+qW3SlXRL71Vqop+6a1SVQxbGaEY1i+TV6qKfumtUlX0S2+VqqInSw+Hq6qiKmPFOGNlHIOVKWGMbaWqGBNlqIYc7vN6sFQVPVmmmWaaqq5HSlUx7KXvfWO6VBWjWrpJFm23tu/ztNNOO+B7+zU9UKqKAWVSjTKp101MaQWvUzEG0t4q7+3jaidFD5aqYkyXgNUKaieAA2xIgCze2xWgta5HiVBVdK1MjIHbzx3MU1uPtb4Dd9ZZZx33+WUve1kz3XTTleOtpEGOmWeeeVxdVMN1M844Y0WaTp+n4FJVDFq6Pem0HzAA1irjATNePf300zezzTZb8/rXv76A5X2xxRZr5p9//uZVr3pVM9dcczUvf/nLm9e85jXl+xvf+MZmhhlmaGaaaaZStK/etUjgmKJdhNGPzxlHxtZtO4xgqSqGrQzFSPFeYCoxtHcFGDnufGDluO+tXs3bX/GKVxTAX/e61zULLLBAqZ9nnnmat7/97c2KK67YzDvvvM2GG27YrLXWWoUsaX+ZZZYp1772ta8t5yIDUswyyyzNK1/5yubVr351IYLPjulD30OZ4xgvVUVXCmO116WekRUGnmOOOYpnMvDss8/evOUtbylAxBu9A807YCPTwObtq666agHpDW94Q/PWt761XL/CCis0O+ywQ1EFoAMb0M5fcMEFm8UXX7x573vfW1TCue985ztLm/PNN18hA9UAuv6MmcLMPffczZxzzjlORaZgIlQVXS+MxWi8mhEZORIMFJLO03mkd0ZGDu8AAzByII3rtJlQwMM322yzAuqBBx7YfOtb32q+/vWvN+94xzsK+Hn/4Ac/WNrS95577tmsu+66BdjddtutKAmPB/yOO+5YwEYodUsuuWTz8Y9/vHn3u99dCOV6RDHOKZQEVUVXS+I24FZeeeViOGR485vfXLwYiIy98MILF091PmAQwzlLL710AYbRtYckCRVve9vbmoMOOqiAu/HGG5f2tbfaaquV8z/1qU81O+20U7Pccss1e+21V7PSSisVAmrzQx/6UOmXlyMWcPWvzy9/+cvleupEOdQttNBC5TvCURp9Jzy0z3mMl6qiayUSDsAkbgxHdtdee+1ifMCTYPH6TW96U5HlTTfdtHjhBz7wgeLdzkechAVlkUUWKX1o17naWn311YsHL7/88qWNb37zm4VM+qUQ22yzTQFUHwik31122aUQYb311isAG8cmm2xSPN15CJdE0znvec97CgEQDgGolHFMQWpQVXSl8Kokcrx/q622aj784Q8XWeVdH/nIR0rixvOQYZ111inn+c5LeR3ZBzTP5IHvete7CsgURaxHivSDFEiQnAJIpJ18IxawgK5toC611FLNFlts0Wy//fYFaON63/veV9p///vfX45Rof3337+QgWrob9llly2EoBL6MhcqYExTCAmqigElS5/JLYzCYIxO6n0GJqPtscceBVTSTZ4BteiiixaPJOMMzpM/8YlPjMsBtttuu3Lc+RJG7SY7DykoBEVAHlLtOkAjBpCRwnGgOS4kUAbfv/jFLxZSAZpKWTmsueaa5bucwXiQUx/Cyeabbz6uL4RTr53xLRk71Y1S+d+Hbg6IEXgZwwOfFzImQBQeSdZ5L89OEkaet9566yK1SCCZ44UMqx3et99++xUghZUs0wAQtQEE5ZC8GQOVUZekkdpo0zlLLLFEIQ9VQAgAUybjRo7PfOYzpW6NNdZoPvvZz5YQosgfjFfo0T9bIrCEs3WDaYyWqmJYSuukeTlQE/cZnTfzFp5PXr/whS80X/rSl5rvfve7BWjGRADg80re6BgF4IXyAQRAGG0yfFYEyQuSc8grkMy1xoAwrkMKbUsS1QEfCRwD+Le//e0SqlZZZZVyrpdwgAhCgdyCQukTgYxBQopocoTsSSDB+JbBY6BUFcNWeCFvBCQSkP0kYDyLp/F8EmodzrORA1iHH354CQ2ketdddy3JGBAQA9DeP/3pTxcl+OhHP1qMzODxfO++65cHZ7sXkfSDFMYlidO2ayR7hx56aEkC995770IGocbKwnUIYqzGJQRo2/UJbcalTcVYXWO+yU3GqBJUFUMqg4UMxxIHeYY1NunlgQBlWFkzA/Ioidfuu+/ebLTRRuV6ck4dxPcjjzyyyD/SuN7aHpA8T9tHHXVUAVAbAFeffQZAaM95JB8I+kQwYPgsD+G54rx6nq1OfiFUUSdebDxUR6w3FopBGbRpngimnaiW89dff/1yrrxDX8JECDqGSlUxWSWyS57FS0RgUJ7A27/yla8UQ/Agss/QW265ZYmrSAJMwMsRZOGnnXZauV4uwKjAYGht8Dik4OGuRRTH9YkMjgORjCdRzKZTxiv+IyU10gag1NkvoC52FvVPEUg+ZeLZzneuNoCKmJJY5EYMpBcqKAFiIaFrxuA+QVUxyYUhGBfrGUesPvHEE5uvfe1rxTAAlzwhBm+6+eabi5dYzjkeDyKrYrU4TKrFfoRCFrmEXTjHnZ+woo14f5I4bVEC0u0648o2s3Ear/qs8z/2sY+VbF9IkcTxcktVxBAWvEvssvXr+qiKsIIY9hrMF3mPPvrocr72KIf2EEv/7bYbxVJVjCuDxaz2EOA7T8qdNx7NMyV7ACLRgBfPjzjiiOZHP/pR2ZwBXHb1AOda/QLPps/xxx9fJDU7gADhUYDMRpF6nooUQDEGBJAn8GaKBCSq4jr9ad949JUtXgqDpDwZyZBCYio0+ay/bD9HYfSpnWOPPbYAfsIJJxSQnX/IIYeUtvQ

In [15]:
// set # of exponential components
param.nComp = 3
globalRslt = op.unary("flim.fitGlobal").input(param).apply()
param.nComp = 1

// showing tau from 0.13 to 0.25
nb.display(fcd.tableDisp(globalRslt, 10, 10))

0.0
Z min = -5.333333492279053
Z max = 3.0639078617095947
A1 min = 0.0
A1 max = 958.3338623046875
Tau1 min = 0.0
Tau1 max = 0.15660516917705536
A2 min = 0.0
A2 max = 990.9171752929688
Tau2 min = 0.0
Tau2 max = 0.1581559181213379
A3 min = 0.0
A3 max = 273.18060302734375
Tau3 min = 0.0
Tau3 max = 1.1072466373443604
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 A3 Tau3 Pseudocolor3 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAtaUlEQVR4Xu3cVawsSxUG4Iu7u17c3d3d3d3d3eHiwSG4huA8kCAhECBIQpDwBgnPvJDwygsPZMhX5/xz1q7unuk9e7Yc9jnJyvRUV1etWv+/pKpnn7POOuusxVGT85///IO2oyLnPve5B22nuQwaDl3Oc57zDNrG5LznPe+gLXKBC1xgdt9jLoOGfZFznetcg7Z1sskzJF46l0ibzkP+D4g1aDh20keL3cpcoh1RGTQM5EIXutCgLbJXD9iL952RrcigYWtSwd0r0FNetq5grM9JDdso4qZ0OU1l0NBkrmcftjG2Aegxl0HDgUgiQtJLjRDnO9/5Bv0vcpGLDNp2I2OEXhc9jokMGs7IluWwo+QaOXExtxIey+WrisReeu9eF8LH5huTuf3OSCfrAAhgF7zgBQf3Vskq1l/4whcetM0BMLqM9aVfP2fV+aIXvWj71Gcu2fdDegfYVMZS2oays2HMuHuVdSTbb9mPNR0F6Qm/oexsmGOsOX2OmmzJWPsqqyLcPsrQQ4+qsTYJe9sIuftpj0td6lKDtnWyjTUVGTTsWTB4E7DIYebndXLAnnlQMmhYKRWgTTyjFoD7YdC+WN1ER7IpgefKlr14LzJoWCkx6KaG7WXKELs9pMk4fTpbJxXosTVpm9JxEzlyEW63BttU5gBKlxohpoxV+1zsYhdr/Xz2/WqEcZ3x0rd/JtGjb9+W7HdU2a2ctM/wxmFKvG3q6Fd7JcDFL37x9jl2thBRaF3hClcYtFcBOm+/5CUvuQTq2te+9o4+YwdeY23blD6l7YMMGnZ460FFiEgObKogBbn0pS+9o7331Fve8pbtE7MBHuNd5zrXWRKqvhH0/I1vfOPFJS5xicF8PnvjX/7yl2+fSIJwiOK69qnS1zj7TZaNpOa3PgeGCFlIf38bYn6ATIXHeHZ08N119PYZw970pjddXOYyl1nc4AY32PEsESk8B3R9tF3lKldphLvsZS/b7l/96ldvz0YXn7FB1q6vTzob75rXvGb7rl8PeC/r7pOe1NuWkXpm2KmXMcVjkFURIjm37zOW22NYEqOSKHyTm9ykXfd5vRLB5+Uud7mmGyDveMc7trbrXve6S+917173utfiete7XtMrZHDfGCFNPBupMl/IIDL0NkkqGlsbmSJ4laln59RPe5AhQHuVVZFirFrXX5g+++yz2/eE60QH19e//vUbuHWcPl0ABTg13+tztatdrV1f9apX3WFk4wFOu+eQwBjXuMY1mide8YpXbFEh/W9zm9u0TwS90pWutLjhDW/Y+o3VIX362LbMIdRMmWbeJjKW5yjbKyz8MjyjARlQjBqC3OIWt9gRFXz3eaMb3WhH3gVS5ujnJfS58pWvPGg3d839QEzkMRYy0MlnxkYM15V4t7rVrVqRaR2pURB4v226DTmp4/DGNiVewXAVCIbLtTAM1Gq0hGQCGJ6pTy0E3RMeE955Ky81NmIANTnVs7yS16d/1a+K+whK32td61otMgE1tQXCGtu9m9/85q2vCJbns45Esp6c2yTHFmTQsFL6ImIqffQ5MiJs9oxm8BjFJ88EYEI3YWwAMnxIYJvGG+X0GFv455Hmdy9n7UkvVfSt6QI56FZ1r2mHiEAhEH3Mm/79r5uSGsiqtBgZI+MByKBhIFNg9mSYkixMGAceEiRH8trk0Vr8xWDxrhRj8UJCL2nDddIFEAL6WEXtmbve9a7LuTMXsiUN0cV3pFNA6uNTG4IiHiJUPXsbGSvksda5tiJjEaLfqlaZQ64VMmjYuvRVbA5Y4ql9f94KyEQXBkQE7a4ZyP49W74YF7kSHYRdBJC3EU4bI9ai0Hi51s933i1d3OEOd2gEcV2jh9qhRj3k9kxSlsglqtCxRoC9yh5BHpWTYw5v7FVqzquKp+Dr+6ffzW52s8V97nOfJSlqXuWNdgK9J9ge1vncZ3gESt9s9QgyIp5o4zqRA4DCuXnyWZ/x+YAHPGCpq89EEKQiQLe+zGuc293udk1H3/vUF1l1mFSl2q6vK/Ygg4aNpA9bFhvjApKnAjaF4BijeVdtryAA8eEPf3jz2Nvf/vbL9oR5xsneni480VxCdwVEmzHUDdqE/KQMBZ1nk55SmOa7vnk+cyKH5+juGhFEE+s3X0hi7WO7IZL5p5xjTMbs18vMreiJiyl2pT1e2QPdS/2BA9BInhVKc80rfE+YZCyhWhilOB0CpvvqBySycP16r6n5vqaVW9/61svrGlEAFSNKJz4BZz65W1vmSlgHcPRFKjoTYEtrvN2aHEmzwyMf+cjlfNaSCBBdx8iwTZmZgk592U2h0kuMbnF3vvOd27UwK3czBqPyHovmQfoDlHEZU3pIARY9tAOA8Xg/YiCCMQJsPCyLBZL72RVEjMNTXWcfn8gQYBAk+dt1xhZF1Bb0Tt1ivOwGQnrP3elOd2rXiGAMUcn3hz70oU2nOFTv9SFDvHYvWMyRsvsZ3qzSR4Y5rM2iAcyr7MsZAwFSaKVvquycydedAFA9H3KlwEOq5PXMhXiJIL5HT8/WqGU95snOJN7IgxEhupm7elDdWWROhaLr6GK9D3zgA5ue97znPZc7gdve9ratLSktxg/Yq0J/PQyL7JUcXS1y4qKGzVxP5ZAoUPMQUHLN4HVPCxgkuMc97rGsAXgTMlggr5VDteunGMw4rmtUYVB65fBIG33vd7/77dCxgmdsOge4bB31MR8dCKAQVbtiNAdR2T0EKNf0AaS+xkcGa6PvK17xisWTn/zkpitiIXXWFAmwY7ugVbLb/lNSnGJ4c12BEe/q35RZLI9lNG0WqWBjsJz1ZwG+p5jSl3HdIwyacE1U//27eQKYbMuQIaDTDxjZMvbPTYlx6jEvSWVfvc6cyIww1kjXRLK73e1ujfDWwRYiCr20OaVkA4SoTtKDOrVbmCNT0WES036yqQH6gfroYEFPf/rTlwv33ds4i+YpWSxDCIeeZ1ggZszkbYD7fMxjHrMEPmf+gDV29vx1ixedavRh8KpnjVR1SwkoOT/PmsOcSE4/3uw78FMkSgt1/FqIijb1JJOdjRnPz9nEHJkEb0Lm9C+p/AToPRHmCs9TkPEABgxQwiyjAqUuttYQ5kUC9+thERIFeGNYEFLVY9uMma1awrW2kAKhaqhVndMz+RwBtFUPFAESiXy3WzGXiGVtOd17ylOesnzdDGj5PzbQplgVDeouBEG0mR8RpYXnPve5y/vEHOuccFtyMtqd+DI16Rib+go7BuUdDMEr8slTgCAMMqpP2yX9FUWezd4aCVXLqcaRoIJBakhnUITK/OYDFhBSzFVd9beeOp5roIaY8Vp68uLUCfogQ8hlreayBqAivHvJ9fSPQ9TzDFGLbdyr0WtOcb1tOYn5zsa5OZPhKJ1jWm3Oxy04XgMEnp0CLpJKX7itBzWERycPI0Yqf2PFYDniTdTwvFoDCECru4yMA5CkG3PEk5O7SdZuDUgFeHne84lsdH/5y1++BPoRj3hEI3iKWIIQdMwJYKICW6UtQpcQMvas9w9AdjZUMCjGKJWd2hIVXOvPmI997GOXz+akTj+GQ4a73/3urS0gxtMS6lXRqbJTpRPbqswHEM/x6pDOM6kntJuvFptYDkzryLkCguU3g57LfEALoWuUkCaMmR+E5KUQndUrPDwpNNtMcxmL7er7hwDsmZweZp5E4f7dyT7LoGGHVEYGoADrmiHr8S62W2wKubvc5S5NUiDxTsbVh6ekr2fve9/7ts9EgJzzGxMoAEag6KSf+56hG6AZPtvDWvDF4+nquZwl1LrC+Dze+BkDUHGA5G/3gSR6kOw+RIX8cOVlL3tZ+7RutYIoaJwQNqTod

### Phasor Analysis

In [16]:
// WIP
param.paramMap = null
phasorRslt = op.unary("flim.fitPhasor").input(param).apply()

org.scijava.ops.flim.FitResults@7ecad72b

## Other settings

### Region of Interest

Sometimes, instead of the whole dataset, only part of the image (e.g. the region near the nucleus) are of our interest. By specifying the `roi` parameter, we neglect unwanted parts outside of it during fitting. This greatly improves the running time on large images.

In [17]:
import net.imglib2.roi.geom.real.OpenWritableBox

min = [ 20, 20 ]
max = [ 100, 100 ]

// define our region of interest, in this case [40, 87] * [40, 87]
roi = new OpenWritableBox([ min[0] - 1, min[1] - 1 ] as double[], [ max[0] + 1, max[1] + 1 ] as double[])

OpenWritableBox [(19.0, 19.0) -- (101.0, 101.0)]

We start the fitting routine the same way as before but with the `roi` parameter:

In [18]:
// fitLMA with roi
lmaRslt = op.binary("flim.fitLMA").input(param, roi).apply()
nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

0.0
Z min = -0.23343004286289215
Z max = 5.321786880493164
A1 min = 0.0
A1 max = 1088.684814453125
Tau1 min = 0.0
Tau1 max = 3.188699722290039
brightness_max automatically set to 1696.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAnpklEQVR4Xu3d5dItR9nG8aAJLsE1WHB3hyDBLbi7u7sTgie46wdcCyuKKj7AMXACFB84iqF+XfXftWp1XuSdvfazN2ul6q6Z1dPTcl/XLd09z85pp5122nKQvZap4CD7JVPBQfZLpoKD7JdMBQfZL5kKDrJfMhUcZL9kKjjIfslUcJD9kqngIPslU8FB9kumgoPsl0wFB9kvmQoOsl8yFRxkv2QqOMh+yVRwkP2SqeAg+yVTwUH2S6aCg+yXTAU7l8te9rLL6aefPu6vcY1rLDe+8Y2Xq13tass1r3nN8eyKV7zictWrXnW5znWus9zwhjccdZRf4QpXGPXufOc7L1e+8pWXS17ykuOdy1zmMssZZ5yx3Oc+91nOOuus5Va3utVypzvdaXnoQx866j7ykY9c7na3uy3nnnvuuF73utddrnKVqyw3uMENhtzhDncYfd3iFrdYzj777OVa17rWcuaZZ4423d/sZjdbbnKTmyxXv/rVx/v61q/6yq90pSuN8RnH9lxPAZkKVsklLnGJqexSl7rUsXugUaQy4AKaopUBDjEADmjPAQNU98qAhyDXvva1BxCXv/zlhwDhpje96XLHO95xXB/xiEcsD3jAA0b755xzzgD0lre85XK9611vechDHjLAU+8ud7nL6Bfw6qqjT+0hpmeut771rcf4jQPQ2rnRjW403kEC4zG3S1/60tP8yUlMjqlglZjoJuDb4tn1r3/9AbgrQFmTK+UCitURwKvvGWVT+qaVst4IFzFqk9Wq671ARQT3rF/b9773vcdz70Q2hNAP0Ik2jFVfrN+7PIZ27nGPewwSIcHNb37zMXakNB5j2Z77SSpTwXEXVs2icvtAo3hAUvBtb3vbYUG3u93thhLJbW5zm2NEoXwWqA5FIwyAgcHVI4n2vedZXsF7vIkygKsDZFYqNACa1QLNGO5+97uPe2QAvuf6vOc97zna5l0ioN+eee8xj3nMGJ/3tK38cpe73LH5/ifCC26XnSCZCo6rpAQWAQy/AcKKxPS73vWuw82yIoBTPNC5Vb8BRyiY1bF8oYDlIYn3gYJU3lEGFGBoR7+ABwrPoI4+xGxK1xYiqYNc97vf/ca7yIMsSIEs+gd85NOOEKM/czEGbXoHicw1UC/O/SPedtkRyVRwXAS43bMKFpk1ApICkYGS7nWvew3r8hsYAAIwhROAqg9spNA262T9Ej9JnjKKJiV5+kOeyAccRGO96gNZP+6BxwPpB8DaRw59FvO1pU9tbYYlBAY6Emnfe+p4rl7z/HegH5EXmAqOi2xOlnK5RJZFgQBlOX5zr0CmRNbHmsRh1siqsn7xVgz2jjLKBZ5M328WTvmBDTQk8BvQroBHoNvf/vajHePwW99IyJXr3/uuxqXvco+8iPp+I7U65sgz6V+/yo3fe8Zg7kixraOTRKaC4yaAN3HWzUJyt5TKyim6uFrCV2YOpJItLli5d9wD0nPWev/733+AycoovyRMW/p//OMfP8BQJiQQ5NMfi0dAwGmD6OfhD3/4ct/73vdYiPG+caivfx7CWFo1aBM5iDEhCW+DXLxC+c+/So6PUKaC1QIEMRnglMICgUGhlEdBpISJG1cOcMrOKoFBkYjyoAc96Jg1arvll0yeZ+ARNl09pRsH61NG+Vy99om2hSOWi5RivffsFXjXmLyfCDO8kH68U84AfM/NT7k2jU09/fIQyOYZEhjPtr4uruwEylSwWgALUJZq0sBmOawJyIBjeSwKgLyA51lz1qMcEZTxCkACdjGcgvXzsIc9bADBK5S9E22U2RebS+Aaq74QEwkAipARSlvGi1jKhKhCF9DEbFcgR+yWotrVFtIar9/6Q5xtfRnbdtkJlKlglZhgu2QUSIDsSolAF3sp5ulPf/pQHOElsh7Asxig2tB58pOfPAABpnYpE5hIIN4iXAmX394DKICQpJUA4Ntl3AxJrUhYaqEG0IA1Zlc5Qkml97RR8qe+9vNwvIfx2HBq1WBs+jDWf7U6OAKZClYJILlKirZMAqbEj/VQGMun8Kc+9amjTH1gU0YxmKJYlnoSM4pUH3l4AgDzJhRJ+ZTL6rjhXDKra3uZ++cRWnmooy4w9QU4V6AaD0+AZOK7dpEA8dp80pZ32ndoU0s9XiyyIr45tWRETnPwvnclysaxrcMTLFPBKqE0CiweAodFZP1l+xTD7SMIJVCwe9ZSCEAcZRToSumkHTsWiVCsWI6AELnTXK223QPJM2RodYB0+mGVrRSMr00jY1fe3kEE8twYHv3oR4+ywpRy40ACXgtJzcdvxDZW9bWp3/9JD4AAwKKgFMWSuXxWQZGRROzOnbIKAMi+2xmkYED7/bKXvWwQItetXVYEvFYPrBi4QGdtrd8RUd/GwLMoM1b9IZR3geh54QGQQDUOz9s+1q+xG1ttmy+vx8qR/a1vfetYSViitmQ0r7aTyw8u7tzkCGQqWCUUaqImR+lAU0ZBlENhJt8GCjfYGhmAQAEARfEQ7d+3n8/qs86SrHIC/QLK+4UEz5AOCGX0iNGGDgttjd+WsSsxFuXaND5eRLl+hCUAc/cIYHwPfOADxzy1qYwAHpHUaxuZZ+ANtXcSkGAqWCUycYC7UhD394QnPGEoIlApY3PNTFgeq+XG/QbyE5/4xPEeIuXy24hhhb2f2wese0ouU1dHmyVsgFeOGN5FEm0hCbLVd8tJz43V85aowDNH5eI80IHJxbtXD8jGzXsgREtP/XuffhjE/3V6eAJlKlglQAcCt22CD37wg5cnPelJw0JZCWAokxVRWtk/BblHAkpiYUBjRay8lQQv4T1EAhIQ5BvuiXsgGotxEH2yYpm8Z676U65tYBiD97UvziMOcFp2dvJnTMZRcgpMbQCf9wC2Oggr9NGDnAeRtXHeeecNozAu9elkcwvYGLZ1umOZClYJZQIScAhAKPxpT3vaUKrYWibvdzt+AGahrnKDZz7zmQMsliah6ozAu6ySe84DsEq/hRXEQAqEI3mMLLowoV2eqHMJ9XgQAJqH3wDXdmGtFQ3Rj7HzNsAtuUN4/ZkH8NvC1hevAHzl6hhfy8oj9ARTwSoBNABN7HGPe9xQAAKwMJNOEYghUXrb2942nkugskbvUxpFA5yyKZSygct6tc8ygcSaAKxtYUM/fVOgjnc3l4XABRhwEUCbrBCJJJTaa7/Ctc0j/SOrPlzNAaDt9nnvsY997Biv38hA1DMP8zYm8yqZNF/9lgv8N0fIx0mmglVCaS1/ACv+s1z3rH7TqrjQV73qVaM+ArTk6vCHMp/73OeO5+qXULJ0wAGLCBveY6GFBX0Am8KBwboLOXmTzhXcq0v5vFBreyC56tc8ZP6dDRhDR8T6AqDw4bdnrogMfASUXyB1h03GQYzVe+ZxRMvCqWCVcPEUZZfPxF/5ylcur3jFK8Z9yydKIPbXJUqUw43KH1xL4sTOzU0YlssiAUUA0G5ewBfL24cwJiCqx9IQ1D1v0crBe65+AwOgRB9IoJ1WC94DsETRODu5dB/Q5mHjiicyXuRhAFYJxmoeLYm1155EW8vbOt2xTAWr5DnPec7yrne961jG+/KXv3xMlkKQgyKA86hHPWp5xjOeMSxEOQVTIpfI+ilL8pi1lFQhBqulyA5svKsegADDIwDP1bOWdK48RUu1lnR+aw8I9WWM3keIZz3rWYMQ5scD8QLmJtzVJ/evXDLIswkTfpvvRz7ykRHuzJlHa3dQXbrRD1KZg7lt63THMhWsEq75+c9//lDas5/97LEpwopL/mTBlOcKYIphOZ4RoaATvkASR3kUysr9Aj5XX8IGNECz2ryCZxSuvK1bY+kMQR3PkUsbLFKSWbueISOyGhMPkkfwjjFrx5W1c/s8m76Q3jxf+MI

In the results above, all other regions outside the box is neglected.

### Binning

Binning settings are enabled by setting the binning kernel parameter. The kernel can be any image. Here we use a square cernel created using the `"create.kernelFlim"` Op, a $3\times3$ image with each pixel valued $\frac{1}{9}$:

In [19]:
kernel = op.unary("create.kernelFlim").input(3).apply()

In [20]:
// spin!
lmaRslt = op.ternary("flim.fitLMA").input(param, roi, kernel).apply()

nb.display(fcd.tableDisp(lmaRslt, 0.13, 0.25))

0.0
Z min = -0.42977118492126465
Z max = 26.733793258666992
A1 min = 0.0
A1 max = 8187.78955078125
Tau1 min = 0.0
Tau1 max = 1.191415548324585
brightness_max automatically set to 13512.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAnaElEQVR4Xu3dY6w0XdbG8Wds27Zt27Zt27Zt256MbWWcsZ3M9/lWk99O/k8qVc+874eevpMz3SdZqepd3Ou6Fvbau/scdthhh0172WlZNexlt2TVsJfdklXDXnZLVg172S1ZNexlt2TVsJfdklXDXnZLVg172S1ZNexlt2TVsJfdklXDXnZLVg172S1ZNexlt2TVsJfdklXDXnZLVg172S1ZNexlt2TVsJfdklXDXnZLVg1blaMc5SjT0Y9+9OkYxzjGdOxjH3s6znGOM534xCcecpKTnGQ66UlPOp3iFKcY21Of+tRje/rTn3461alONeR0pzvdkNOe9rTTyU52snHN8Y9//OmMZzzjdLaznW26wAUuMF3kIheZrnzlK09XvOIVp6tc5SrTJS5xiekyl7nMdMELXnDs25Lzne980/nPf/5xvmec85znHFvPPfnJTz72z3CGM4znaTvrWc86nqOtdznNaU4zjp3oRCca/dG3ox71qEfYb3LkIx95dWwuRzrSkVZtW5ZVw9ZE5wAPsBOc4ARDaQTgc6FUAFP0mc50punMZz7zAPcsZznL2NcGHOe6/oQnPOG45uxnP/sgwCUveclBgCtd6UqDBBe72MWmy1/+8tOlL33pQQjnaEMA+xe+8IXHvS90oQtN5zjHOQbQnuGZPhP7COIZ5z73uUcb8mg717nONZ6PjMc61rFGH5EgMG0J8G0R4T8BjUDLti3LqmEjqbPLdp3X8Sw+6znlKU85tiwLCNoBT+GUDQhb4LBa4FC+a5zLUvMESOF8wACVABhItoENQAI49yb2z3Oe84xzPYO4D5KQQHc8IrgmEjnufRDyeMc73vAGeYL0EQGOecxjDl3YzvWm7Yi8x5Zl1bCRxPR5m06xCh2mHC6TMimQogPIZyRAhizLeYABPhJoRw6eAAi5ZESKLK5xHxKY9gPSeRGQp0Ec9/QcBPHcQgO56EUverjHIN5B26Uudanpcpe73HTxi198XOOekZGH4w1YdK4/IvR5rqf0ttTdIZBVw39VdOxoRzvasHxKYbHAC3hgUTqFnve85z08lgMDCACjdMpGgLnbdR3lI497AtSWFMsRwz7xTGRxHqDEcWHEee7p/p6FRN4F+O4vpCCgZyOCZ3uvwo3zCgPui+RIIDSRORHm3mAOtn16cs5Sh1uWVcN/ReYujQIohIKAQLkURoH2xefLXvayQ7m5WARwPEsjEjhtuV2fXescoLHmvEOJIwIUVngB9/UOzitkEO1yBGAC2mfgIpkEsucWFmx7f5IXKpnVV4Jgcp5IQB/AXrp6+tK2bD8Esmr4r0iMZvniIsunEBbPegKZMik96ytBAwaAAeB4BMiNOw8wV7jCFcb5QDViYIFAyMJLHG0BXpIXeFmze0saPYfVI5X7e36hhxfQrg2B8ibu10hBPxsxEKTkBehh7gWy/nm+dETEOASyathY5kM8QLACCgIAhSdAAC7F5s4BwBsAhLId9/mqV73q2LoOGEB1jvNdC4yGjwjHClkkEAoH3iHLdW/7COEZCHed61xngB4RxHfnIIDzeQT7iItQwPc890W2rJ5Hmec4SOC84x73uEM3czdfcrwkxiGUVcNGoiM6Kt7rtM4DC2gsLddeRs0Ka4sIBLgIARwWbjgHGB5AnuC+wHGec9wPCbRnhcVy1loYcF4W7RggezfPIUjg2QhXAooQnu8YYN0L6MjVSKIhqucR7+oZnomMdCIR5hnTF4uvLvKfRlBbllXDRsLyAQAIyg2EFJp7LbsusQt47SWAPlM8N3+ta11ruva1rz0AomTPQARgFNeBMi/OOF4uwBM5BgzvVHvuurBS9l+C593zNDyC93Xf4rv76GfXeufyB2Txvq5BDu9QPgBw+kIAhCDLxPAQyaphIwn4rKok6upXv/p0tatdbSRtijE+c+sUS8msheICUxu37Lyb3OQm041udKPpBje4wbgXRTdaqH5gW5gh9im92Azs3i3rBWAJoXd1zNazvXdJIALkHXq/OckKI94NSYi+en/9a+SCeJ7LMyICY0GEsv/CwVKnW5ZVw0ZCgY3ZbQGuKnfNa15zuFBCOeItq87NNrSzXxGI4lk98J2POI3Nnc/qcsEAsU+5WXyVQvkAsObncMm58IakZfQAK+FrKNjwEHkQK6I0YnB+ngphr3GNa4y+apvnKvpYmEICIaGRwf8EAcroG04BGAEohMXbB+SNb3zj6frXv/4gCGWWmRcSiHMR4GY3u9l03eted9wjcCjStqIRcFm7EQDggV5Rpky9EnKjhDxC7t/9gNp79JzCBvDdYx7SeIlqBUh+i1vcYrrPfe4z3e1udxvvnbdzDBncE8GbPygnKAk88DkAt4fpKZE7zANQBsBLqCgQUVhUQLImbRQaYSiwOgEgqvAhQ5W/LHpOAJbe8LMQ4XPnug6gCBAJbN23oWqVRsRxH88GOEJe73rXG/0BrP7c/OY3n25zm9tMd7nLXaZb3epWwxPosz7weuVA7u29vYuEeVkgWup0y7Jq2Eh0GMt1lHK4RO47Fy5ONjvnHMpOuRRNMYCgUB6C+7d1LZcc+IWMyrqUyaKAXnwuKUSqJpUqDFWTKElFVs9tmKm9HMP72Y90SM4zESSd5yl3vvOdBxFuetObHm71jAChqzHY9yzv433nY/8D7wGqzOkkBd3ylrccykCAeTnXtsQLGCVllC3eUizgEYg0OmiYVQzOShEg6wc+ca+SxUYkDUm1ASEiVSCal5FdSzyrIhXRR+GIu4/YPgd2HoHVG75Ww4gs2puVdH9egOVXDVzqdMuyathIdI4SdJyCuMLb3e52I4OnuDzAvKZOuQAgyJH1uwb4FFoCBaBqCE3MNKQT04EI7EYB83E5AuXaI5G2rN51VfN4irxDIYkgdwADHQEASzov0PNcztfv2niKzve+vECjgQNPAAoBnC23yB3qMHdIOSyh+r59CuIpuNXChut5DtuUizBZfHHfPpADuvwBsIAEYiOKrrV1HvLMq4HO5T14ER7JPT0TWQHoXYFoe8Mb3nAQADkRnlVX30B+fdV/ia5cwTmdr6+OV+0sBPIC6gNkqdMty6phI2m8ruPktre97fACrIVydFrnAU2Rt771rac73vGOh1uS81xPebLoyEPBzRfIMYrRxWdWjGAVa4Da5FBjfe4WUZzrPgBGFscamrUSyDnVMCRxva9w5j1ZOQF+7pzUBvSGuz7nNVxrvwIRUnp2BSK1gaVOtyyrho1EZszqCSu+053uNEJABGj4RCm3v/3tp3ve854ja3Y+oCmo5BEREIDVzCdpWE3xuto7RWp33L62sv+A5+KBjUCNUgBdyCh/8LliEI+CAEADqP7pB49FtFfMQhZ9cE6Jrz7whMKhvjEAz29GEVGFLx4A+AeeAEAHrE4bEt3hDncYIFOK+Ed5FME7IIcxsy1rB3oVQscjQBaTywRarh1YDeWAYFtFsHG7fW3OjyTAd371eoRyjvOdV8KIBN5nvsSMpwn4wBdWvJ9z9T3gecB73OMeY3TAg5QHeX5k9X4tbbOWYKnTLcuqYSPhzu9973tPd73rXacHPvCB0/3vf//pQQ960OHKyEMIC5QC/MjB0gGPBMjAmoqfACAAnLt1ArS5m2dRJYWOIYJ9igYUUAFQ6GjsDwz3YZWIRRDPu3gvrr6SMCIXxysi6QPC0wHy6+9DH/rQ6WEPe9h03/ved5C8fvAernedd5SDNFew1OmWZdWwkWC7Khhwdfpe97rXaOMCeQLK4fKFBcqiFMALEcAGun1trIzY504dZ21l9sCqFlBiqJ0lIwIAtRUqkMD5AGP52u1XXm5EUPkXSQDFqnsf1t/iUvlAYLrGO+oX4uv3Qx7ykOnxj3/89KhHPWp8lu8gUsN